# Import Libraries

In [11]:
import pandas as pd
import numpy as np

import snowflake.connector
from snowflake.snowpark import Session
from snowflake.snowpark.functions import month,year,col,sum
from snowflake.connector.pandas_tools import write_pandas
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import serialization
import os
import re

In [12]:
# formatting
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
# warning messages
# import warnings
# warnings.filterwarnings("ignore")

# Data Connections

In [13]:
# Set the connection parameters using 'SVC_BI'
connection_parameters = {
                'user': 'hidden', # snowflake service account
                'password': 'hidden', # The password for the Snowflake account
                'account': 'hidden' # The Snowflake account name
}

# Define a function to get data from a Snowflake table
def get_table_data(table_name):
    """
    This function gets data from a Snowflake table and returns it as a pandas dataframe.
   
    Parameters:
    table_name (str): The name of the table to get data from.
   
    Returns:
    pandas.DataFrame: The data from the specified table.
    """
    # Create a new session using the connection parameters
    new_session = Session.builder.configs(connection_parameters).create()
    # Get the data from the specified table
    df = new_session.table(table_name) # add .limit(10) if you want to restrict the number of rows
    # Return the data as a pandas dataframe
    df = df.to_pandas()
    return df

In [8]:
# import warranty parts data
df_warranty_claims_parts = get_table_data('INGEST_DB.RAW_US_D2.DBO_WARRANTY_CLAIMSDATA_PARTS')

In [9]:
df_warranty_claims_parts.shape

(10, 41)

In [5]:
# import warranty servicer data
df_warranty_claims = get_table_data('INGEST_DB.RAW_US_D2.DBO_WARRANTY_CLAIMSDATA')

In [15]:
# import serial number data
df_serial = get_table_data('INGEST_DB.RAW_US_D2.DBO_SER_SERIAL')

In [8]:
# delete later
df_serial.to_csv('serial_numbers.csv')

In [10]:
# import invoice data
df_invoice = get_table_data('GDW_DB.DATAMART.V_INVOICE_LINE_FACT')

In [11]:
# import AP data
df_ap = get_table_data('INGEST_DB.RAW_US_D2.DBO_AP_PAYMENT')

In [12]:
# import sales orders
df_sales_order = get_table_data('GDW_DB.DATAMART.V_SALES_ORDER_LINE_FACT')

In [13]:
# import distributor credits
df_distributor_credits = get_table_data('FIN_DB.WARRANTY.FACT_DISTRIBUTOR_CREDITS')

In [14]:
# import warranty_claimsdata_parts_shipments
# this table was an addition made in September 2022
# invoices flow to this table after late September 2022 instead of the original table, df_warranty_claims_parts
# this table also provides additional tracking fields
# this table  will need to be appended to df_warranty_claims_parts

df_warranty_claimsdata_parts_shipments = get_table_data('INGEST_DB.RAW_US_D2.DBO_WARRANTY_CLAIMSDATA_PARTS_SHIPMENTS')

In [15]:
# import company_dim
df_company = get_table_data('GDW_DB.DATAMART.V_COMPANY_DIM')

In [9]:
# import product
df_product = get_table_data('GDW_DB.DATAMART.V_ITEM_DIM')

# Build Data Model

# Warranty Claim Parts

In [17]:
# convert data type
df_warranty_claims_parts['INVOICEID'] = ( df_warranty_claims_parts['INVOICEID'].fillna(0).
                                         astype(str).replace(r'\.0$', '', regex=True)).astype('Int32')

In [18]:
# warranty parts
df_warranty_claims_parts.tail()

,CLAIMPARTID,CLAIMID,POSITIONID,PARTNUMBER,PARTDESC,PARTQTY,PARTREPLACE,PARTCOVERED,PARTSHIP,DENIALCODE,PARTBARCODEPRINTEDDATE,PARTSCANNEDDATE,PARTCONDITION,PARTCOMMENT,DATESUBMITTED,DATEUPDATED,DELETED,SHIPREQUESTED,SHIPORDERNO,SCANNEDBYID,PARTNOSHIPPED,SHIPFINISHED,SPIFF,PAYSPIFF,APEXPORTED,INVOICEID,TRACKING_1,TRACKING_2,SHIPMENT_DATE,SHIPMENT_CODE,ORDERDATE,PRESHIPPEDFLAG,REMAINUNIT,PARTSNUMBER,APPROVEDSPIFF,CREDIT,ISCREDITEXPORTED,DISPLAYMSG,LISTPRICE,AUDIT_SHA2_PK,AUDIT_TIMESTAMP
1339171,207984,153472,1,AX6000MA3,MANIFOLD,1.00,True,True,True,,NaT,NaT,None,None,2009-01-22 17:23:02.263,NaT,False,True,1-332630,NaN,AX6000MA3,False,NaN,False,False,11505429,446269074123208,446269074123321,2009-01-30,579.00,2009-01-28,False,0,None,NaN,NaN,False,None,NaN,b'\xf2\xd7Q\xac\xb3\x88\x83\xbb:Ykg\x95\xae\xb...,2024-01-16 09:54:37.285000-05:00
1339172,207985,153473,1,AXV621D,KIT-A-FRAME,1.00,True,True,True,,NaT,NaT,None,None,2009-01-22 17:23:15.420,NaT,False,True,1-331917,NaN,AXV621D,False,NaN,False,False,11504248,446269074105662,,2009-01-26,579.00,2009-01-23,False,0,None,NaN,NaN,False,None,NaN,b'Js\xd3\xaf\x1a\x9f\xca\x99`.\x11{Yp\xe0(\xd5...,2024-01-16 09:54:37.285000-05:00
1339173,207986,153473,2,AXV055P,MAIN TURBIN BEARINGS,1.00,True,True,True,,NaT,NaT,None,None,2009-01-22 17:23:15.483,NaT,False,True,1-331917,NaN,AXV055P,False,NaN,False,False,11504248,446269074105662,,2009-01-26,579.00,2009-01-23,False,0,None,NaN,NaN,False,None,NaN,b'=\x13k\xb7c\x86\x1d\x07\x91\r\xe4+\n.\nM\x1f...,2024-01-16 09:54:37.285000-05:00
1339174,207987,153474,1,AXV055P,TURBINE BEARING,1.00,True,True,True,,NaT,2009-03-09 14:56:03.343,0,,2009-01-22 17:28:43.893,NaT,False,True,1-331938,19.00,AXV055P,False,NaN,False,False,11504261,446269074106232,,2009-01-26,579.00,2009-01-23,False,0,,NaN,NaN,False,None,NaN,"b'\xea\x8f\xb8\xa3\xa5\xcf\xc0\xfb\xeb,9jdH\xc...",2024-01-16 09:54:37.285000-05:00
1339175,207988,153474,2,AXV417WHP,POD KIT,1.00,True,True,True,,NaT,2009-03-09 14:56:39.967,0,,2009-01-22 17:28:43.987,NaT,False,True,1-331938,19.00,AXV417WHP,False,NaN,False,False,11504261,446269074106232,,2009-01-26,579.00,2009-01-23,False,0,,NaN,NaN,False,None,NaN,b'\x980\xebDc7\x98Uj\x80\xb3T\x7f~b\x81s\xe2\x...,2024-01-16 09:54:37.285000-05:00


In [19]:
# remove unnecessary columns
columns_to_drop = ['PARTBARCODEPRINTEDDATE', 'PARTSCANNEDDATE', 'SCANNEDBYID', 'DISPLAYMSG', 'AUDIT_SHA2_PK', 'AUDIT_TIMESTAMP']

df_warranty_claims_parts.drop(columns=columns_to_drop, inplace=True)

In [20]:
df_warranty_claims_parts.shape

(1339176, 35)

In [21]:
df_warranty_claims_parts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339176 entries, 0 to 1339175
Data columns (total 35 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   CLAIMPARTID       1339176 non-null  int32         
 1   CLAIMID           1339176 non-null  int32         
 2   POSITIONID        1339176 non-null  int8          
 3   PARTNUMBER        1339176 non-null  object        
 4   PARTDESC          1339176 non-null  object        
 5   PARTQTY           1338351 non-null  float64       
 6   PARTREPLACE       1339176 non-null  bool          
 7   PARTCOVERED       1339176 non-null  bool          
 8   PARTSHIP          1339176 non-null  bool          
 9   DENIALCODE        1328681 non-null  object        
 10  PARTCONDITION     16002 non-null    object        
 11  PARTCOMMENT       16002 non-null    object        
 12  DATESUBMITTED     1339176 non-null  datetime64[ns]
 13  DATEUPDATED       58924 non-null    dateti

In [22]:
# create 'SO_ID' to allow for joining to other tables later on
df_warranty_claims_parts['SO_ID'] = df_warranty_claims_parts['SHIPORDERNO'].str.split('-').str[1]

# Convert to numeric, coercing errors to NaN, then fill NaN with 0
df_warranty_claims_parts['SO_ID'] = pd.to_numeric(df_warranty_claims_parts['SO_ID'], errors='coerce').fillna(0).astype('Int32')

# confirm split works
df_warranty_claims_parts['SO_ID'].value_counts(dropna=False)

SO_ID
0         265018
705035       205
607043       192
600045       166
567674       144
           ...  
614003         1
614602         1
195617         1
489423         1
550844         1
Name: count, Length: 357755, dtype: Int64

In [23]:
# create 'COMPANY_ID' to allow for appending distributor credits later on
df_warranty_claims_parts['COMPANY_ID'] = df_warranty_claims_parts['SHIPORDERNO'].str.split('-').str[0]

# Convert to numeric, coercing errors to NaN, then fill NaN with 0
df_warranty_claims_parts['COMPANY_ID'] = pd.to_numeric(df_warranty_claims_parts['COMPANY_ID'], 
                                                       errors='coerce').fillna(0).astype('Int32')
# confirm split works
df_warranty_claims_parts['COMPANY_ID'].value_counts(dropna=False)

COMPANY_ID
1         435988
6         289984
0         265017
9         216487
60         99662
7          31992
15            45
534869         1
Name: count, dtype: Int64

In [24]:
df_warranty_claims_parts.head()

,CLAIMPARTID,CLAIMID,POSITIONID,PARTNUMBER,PARTDESC,PARTQTY,PARTREPLACE,PARTCOVERED,PARTSHIP,DENIALCODE,PARTCONDITION,PARTCOMMENT,DATESUBMITTED,DATEUPDATED,DELETED,SHIPREQUESTED,SHIPORDERNO,PARTNOSHIPPED,SHIPFINISHED,SPIFF,PAYSPIFF,APEXPORTED,INVOICEID,TRACKING_1,TRACKING_2,SHIPMENT_DATE,SHIPMENT_CODE,ORDERDATE,PRESHIPPEDFLAG,REMAINUNIT,PARTSNUMBER,APPROVEDSPIFF,CREDIT,ISCREDITEXPORTED,LISTPRICE,SO_ID,COMPANY_ID
0,1480228,1225821,1,RCX97400,MOTOR ASSY - SV/EV,1.00,True,True,True,None,None,None,2024-07-28 14:42:12.547,NaT,False,False,None,None,False,40.00,True,False,0,None,None,NaT,NaN,NaT,None,None,None,40.00,0.00,False,663.24,0,0
1,1480210,1225801,3,LMCUS11050,UCL COLORLOGIC 80 ACCENT LIGHT,1.00,True,True,True,None,None,None,2024-07-28 11:36:01.587,NaT,False,False,None,None,False,20.19,False,False,0,None,None,NaT,NaN,NaT,None,None,None,NaN,0.00,False,244.77,0,0
2,1480199,1216739,1,SPX3206Z1VSPE,PWR END-TRISTAR VS,1.00,True,True,True,None,None,None,2024-07-28 10:35:39.767,NaT,False,False,None,None,False,40.00,True,False,0,None,None,NaT,NaN,NaT,None,None,None,40.00,0.00,False,"3,094.28",0,0
3,1480247,1225836,1,LACUS11100,100' COLORLOGIC 320 ACENT LIGHT,1.00,True,True,True,None,None,None,2024-07-28 17:29:11.997,NaT,False,False,None,None,False,0.00,False,False,0,None,None,NaT,NaN,NaT,None,None,None,NaN,0.00,False,833.15,0,0
4,1480241,1225828,1,FDXLGSV0001,GAS VALVE NAT. KIT FD,1.00,True,True,True,None,None,None,2024-07-28 16:04:32.733,NaT,False,False,None,None,False,34.89,True,False,0,None,None,NaT,NaN,NaT,None,None,None,34.89,0.00,False,422.88,0,0


# Warranty Claim Parts Shipments

In [25]:
df_warranty_claimsdata_parts_shipments.head()

,CLAIMSHIPMENTID,CLAIMPARTID,WEBCLAIMID,PARTNUMBER,INVOICEID,SHIPMENT_CODE,SHIPMENT_DATE,TRACKING_1,TRACKING_2,TRACKING_3,TRACKING_4,TRACKING_5,TRACKING_6,TRACKING_7,IMPORT_DATE,AUDIT_SHA2_PK,AUDIT_TIMESTAMP
0,110599,1407199,W1162398,SMX11022320,13088474,201,2024-01-24,1Z057E130320045665,,,,,,,2024-01-25 03:30:03.487,"b'\xf4\xe67r\x08\x8b\xe1\xe2\x8d\xef5Z8""\x15\x...",2024-01-25 03:39:13.125000-05:00
1,110789,1407494,W1162638,SP071621,600297081,201,2024-01-24,1Z9W3Y810320050984,,,,,,,2024-01-25 03:30:03.617,b'{b\x04z\xb9\xc1\x83\xd5xSa\xab\nO\xa0\x1c$S\...,2024-01-25 03:39:13.125000-05:00
2,110376,1391137,W1148194,TCELL940-W,600297020,201,2024-01-24,1Z9W3Y810320051778,,,,,,,2024-01-25 03:30:03.343,b'\xd0a\xd9\xd9\x06)\x18U\xc8\xc9U\x94h\xddi+N...,2024-01-25 03:39:13.125000-05:00
3,110802,1407511,W1162655,PVLHP1900GRPK10,600297076,201,2024-01-24,1Z9W3Y810320051107,1Z9W3Y810320051116,,,,,,2024-01-25 03:30:03.620,"b""n\x82\xdczn\xa8*\x025\xf5f<\x1f\xcf@;\xe83;g...",2024-01-25 03:39:13.125000-05:00
4,110434,1406418,W1161724,LPCUN11100,600297060,201,2024-01-24,1Z9W3Y810320051287,,,,,,,2024-01-25 03:30:03.390,b'\xba\xa0\xba\xf3C>\x7f;N\xe2\xee\x8b\x18\x12...,2024-01-25 03:39:13.125000-05:00


In [26]:
# remove unnecessary columns
columns_to_drop = ['IMPORT_DATE', 'AUDIT_SHA2_PK', 'AUDIT_TIMESTAMP']

df_warranty_claimsdata_parts_shipments.drop(columns=columns_to_drop, inplace=True)

In [27]:
# rename columns that are also in df_warranty_claims_parts so data can be appended
df_warranty_claimsdata_parts_shipments.rename(
                                        columns={"INVOICEID":"INVOICEID_VERSION_1"},
                                        inplace=True)

df_warranty_claimsdata_parts_shipments.rename(
                                        columns={"TRACKING_1":"TRACKING_1_VERSION_1"},
                                        inplace=True)

df_warranty_claimsdata_parts_shipments.rename(
                                        columns={"TRACKING_2":"TRACKING_2_VERSION_1"},
                                        inplace=True)

df_warranty_claimsdata_parts_shipments.rename(
                                        columns={"PARTNUMBER":"PARTNUMBER_VERSION_1"},
                                        inplace=True)

df_warranty_claimsdata_parts_shipments.rename(
                                        columns={"SHIPMENT_DATE":"SHIPMENT_DATE_VERSION_1"},
                                        inplace=True)

df_warranty_claimsdata_parts_shipments.rename(
                                        columns={"SHIPMENT_CODE":"SHIPMENT_CODE_VERSION_1"},
                                        inplace=True)

In [28]:
# format
df_warranty_claimsdata_parts_shipments['INVOICEID_VERSION_1'] = ( 
                            df_warranty_claimsdata_parts_shipments['INVOICEID_VERSION_1'].astype('Int32')
)

df_warranty_claimsdata_parts_shipments['CLAIMSHIPMENTID'] = ( 
                            df_warranty_claimsdata_parts_shipments['CLAIMSHIPMENTID'].astype('Int32')
)

In [29]:
# Need to keep track of original claims table vs. new shipments table when appending
df_warranty_claimsdata_parts_shipments['D2_VERSION'] = 'Version_1'

In [30]:
df_warranty_claimsdata_parts_shipments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159146 entries, 0 to 159145
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CLAIMSHIPMENTID          159146 non-null  Int32         
 1   CLAIMPARTID              159146 non-null  int32         
 2   WEBCLAIMID               159146 non-null  object        
 3   PARTNUMBER_VERSION_1     159146 non-null  object        
 4   INVOICEID_VERSION_1      159146 non-null  Int32         
 5   SHIPMENT_CODE_VERSION_1  159146 non-null  int16         
 6   SHIPMENT_DATE_VERSION_1  159146 non-null  datetime64[ns]
 7   TRACKING_1_VERSION_1     159146 non-null  object        
 8   TRACKING_2_VERSION_1     159146 non-null  object        
 9   TRACKING_3               159146 non-null  object        
 10  TRACKING_4               159146 non-null  object        
 11  TRACKING_5               159146 non-null  object        
 12  TRACKING_6      

In [31]:
# is claimPartID unique in df_warranty_claimsdata_parts_shipments?
print('Total Unique Records:', df_warranty_claimsdata_parts_shipments['CLAIMPARTID'].nunique())
print('Total Length of Records:', len(df_warranty_claimsdata_parts_shipments['CLAIMPARTID']))
print('Is field unique:', df_warranty_claimsdata_parts_shipments['CLAIMPARTID'].is_unique)

Total Unique Records: 158289
Total Length of Records: 159146
Is field unique: False


In [32]:
# is claimPartID unique in df_warranty_claims_parts?
print('Total Unique Records:', df_warranty_claims_parts['CLAIMPARTID'].nunique())
print('Total Length of Records:', len(df_warranty_claims_parts['CLAIMPARTID']))
print('Is field unique:', df_warranty_claims_parts['CLAIMPARTID'].is_unique)

# therefore, there is a one to many relationship from
#     df_warranty_claims_parts to df_warranty_claimsdata_parts_shipments

Total Unique Records: 1339176
Total Length of Records: 1339176
Is field unique: True


In [33]:
# join
df_warranty_claims_parts_all = pd.merge(df_warranty_claims_parts, 
                                            df_warranty_claimsdata_parts_shipments, 
                                            on='CLAIMPARTID', how='left')

# D2_Version with NaN is Version_0
df_warranty_claims_parts_all['D2_VERSION'].fillna('Version_0', inplace=True)

C:\Users\mherndon\AppData\Local\Temp\ipykernel_13300\196199995.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_warranty_claims_parts_all['D2_VERSION'].fillna('Version_0', inplace=True)


In [34]:
df_warranty_claims_parts_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340033 entries, 0 to 1340032
Data columns (total 51 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   CLAIMPARTID              1340033 non-null  int32         
 1   CLAIMID                  1340033 non-null  int32         
 2   POSITIONID               1340033 non-null  int8          
 3   PARTNUMBER               1340033 non-null  object        
 4   PARTDESC                 1340033 non-null  object        
 5   PARTQTY                  1339208 non-null  float64       
 6   PARTREPLACE              1340033 non-null  bool          
 7   PARTCOVERED              1340033 non-null  bool          
 8   PARTSHIP                 1340033 non-null  bool          
 9   DENIALCODE               1329538 non-null  object        
 10  PARTCONDITION            16003 non-null    object        
 11  PARTCOMMENT              16003 non-null    object        
 12  

In [35]:
# save memory
del df_warranty_claims_parts
del df_warranty_claimsdata_parts_shipments

In [36]:
# Update original columns with 'VERSION_1' colulmns when 'D2_VERSION' equals 'Version_1'
df_warranty_claims_parts_all.loc[( df_warranty_claims_parts_all['D2_VERSION'] == 
                                     'Version_1', 'INVOICEID' )] = (
                                    df_warranty_claims_parts_all['INVOICEID_VERSION_1'] )

df_warranty_claims_parts_all.loc[( df_warranty_claims_parts_all['D2_VERSION'] == 
                                     'Version_1', 'PARTNUMBER' )] = (
                                    df_warranty_claims_parts_all['PARTNUMBER_VERSION_1'] )

df_warranty_claims_parts_all.loc[( df_warranty_claims_parts_all['D2_VERSION'] == 
                                     'Version_1', 'TRACKING_1' )] = (
                                    df_warranty_claims_parts_all['TRACKING_1_VERSION_1'] )

df_warranty_claims_parts_all.loc[( df_warranty_claims_parts_all['D2_VERSION'] == 
                                     'Version_1', 'TRACKING_2' )] = (
                                    df_warranty_claims_parts_all['TRACKING_2_VERSION_1'] )

df_warranty_claims_parts_all.loc[( df_warranty_claims_parts_all['D2_VERSION'] == 
                                     'Version_1', 'SHIPMENT_DATE' )] = (
                                    df_warranty_claims_parts_all['SHIPMENT_DATE_VERSION_1'] )

df_warranty_claims_parts_all.loc[( df_warranty_claims_parts_all['D2_VERSION'] == 
                                     'Version_1', 'SHIPMENT_CODE' )] = (
                                    df_warranty_claims_parts_all['SHIPMENT_CODE_VERSION_1'] )

In [37]:
# remove unnecessary columns
columns_to_drop = ['INVOICEID_VERSION_1', 'PARTNUMBER_VERSION_1', 'TRACKING_1_VERSION_1', 'TRACKING_2_VERSION_1', 
                   'SHIPMENT_DATE_VERSION_1', 'SHIPMENT_CODE_VERSION_1']

df_warranty_claims_parts_all.drop(columns=columns_to_drop, inplace=True)

In [38]:
# create a key for joining with invoices later on
df_warranty_claims_parts_all['INVOICEID_SO_ITEM'] = ( 
    df_warranty_claims_parts_all['INVOICEID'].astype(str) 
    + '_' 
    + df_warranty_claims_parts_all['SO_ID'].astype(str)
    + '_' 
    + df_warranty_claims_parts_all['PARTNUMBER'].astype(str) )

# confirm new key format
df_warranty_claims_parts_all['INVOICEID_SO_ITEM'].value_counts(dropna=False)

INVOICEID_SO_ITEM
0_0_GLX-CELL-15-W              9137
0_0_AXV621DAT                  8049
0_0_GLX-PCB-MAIN               7528
0_0_GLX-PCB-RITE               7103
0_0_AXV417WHP                  6544
                               ... 
61369950_664672_AXV055P           1
61369706_664686_FDXLICB1930       1
61369035_663797_GVA-24            1
61370167_664989_GLX-PCB-DSP       1
11504248_331917_AXV055P           1
Name: count, Length: 837812, dtype: int64

In [39]:
# create a key for joining with sales later on
df_warranty_claims_parts_all['SO_COMPANY_ITEM'] = (
        df_warranty_claims_parts_all['SO_ID'].astype(str)
        + '_' 
        + df_warranty_claims_parts_all['COMPANY_ID'].fillna(0).astype(float).round(1).astype(str)
        + '_'
        + df_warranty_claims_parts_all['PARTNUMBER'].astype(str) )

In [40]:
# validate keys look ok
df_warranty_claims_parts_all[df_warranty_claims_parts_all['D2_VERSION'] == 'Version_1'].tail()

,CLAIMPARTID,CLAIMID,POSITIONID,PARTNUMBER,PARTDESC,PARTQTY,PARTREPLACE,PARTCOVERED,PARTSHIP,DENIALCODE,PARTCONDITION,PARTCOMMENT,DATESUBMITTED,DATEUPDATED,DELETED,SHIPREQUESTED,SHIPORDERNO,PARTNOSHIPPED,SHIPFINISHED,SPIFF,PAYSPIFF,APEXPORTED,INVOICEID,TRACKING_1,TRACKING_2,SHIPMENT_DATE,SHIPMENT_CODE,ORDERDATE,PRESHIPPEDFLAG,REMAINUNIT,PARTSNUMBER,APPROVEDSPIFF,CREDIT,ISCREDITEXPORTED,LISTPRICE,SO_ID,COMPANY_ID,CLAIMSHIPMENTID,WEBCLAIMID,TRACKING_3,TRACKING_4,TRACKING_5,TRACKING_6,TRACKING_7,D2_VERSION,INVOICEID_SO_ITEM,SO_COMPANY_ITEM
1201715,1467765,1214896,1,005402250500,PARAMOUNT CHECK VALVE,1.00,True,True,True,,None,None,2024-07-09 14:55:23.060,NaT,False,True,60-411829,,False,0.00,False,False,600339052,1Z9W3Y810320926190,,2024-07-17,108.00,2024-07-15,False,0,None,0.00,0.00,False,47.84,411829,60,155526,W1214896,,,,,,Version_1,600339052_411829_005402250500,411829_60.0_005402250500
1201716,1467596,1214753,1,TCELL940-W,,1.00,True,True,True,,None,None,2024-07-09 11:33:51.470,NaT,False,True,1-158128,,False,40.00,True,True,13150538,1Z057E130320977257,,2024-07-24,201.00,2024-07-15,False,0,None,40.00,0.00,False,"1,975.68",158128,1,158140,W1214753,,,,,,Version_1,13150538_158128_TCELL940-W,158128_1.0_TCELL940-W
1201717,1467781,1214909,1,FDXLBWR1930,BLOWER ASSY,1.00,True,True,True,,None,None,2024-07-09 15:14:59.350,NaT,False,True,1-158166,,False,33.09,True,True,13149815,1Z057E130320949215,1Z057E130320949617,2024-07-23,201.00,2024-07-15,False,0,None,33.09,0.00,False,401.15,158166,1,157552,W1214909,,,,,,Version_1,13149815_158166_FDXLBWR1930,158166_1.0_FDXLBWR1930
1201718,1466603,1213943,1,FDXLPRM1930,RETURN MANIFOLD,1.00,True,True,True,,None,None,2024-07-06 14:01:53.487,NaT,False,True,7-429148,,False,8.11,True,True,70390620,1Z8W406V2044011154,,2024-07-19,202.00,2024-07-15,False,0,None,8.11,0.00,False,120.15,429148,7,156346,W1213943,,,,,,Version_1,70390620_429148_FDXLPRM1930,429148_7.0_FDXLPRM1930
1297753,976849,776609,1,TCELL925-W,,1.00,True,True,True,,None,None,2019-09-13 13:20:36.877,NaT,False,True,9-579812,,False,40.00,True,True,90693411,1Z0455980344439649,,2023-06-26,201.00,2023-05-17,False,0,None,40.00,0.00,False,NaN,579812,9,52292,W776609,,,,,,Version_1,90693411_579812_TCELL925-W,579812_9.0_TCELL925-W


In [41]:
# validate keys look ok
df_warranty_claims_parts_all[df_warranty_claims_parts_all['D2_VERSION'] == 'Version_0'].head()

,CLAIMPARTID,CLAIMID,POSITIONID,PARTNUMBER,PARTDESC,PARTQTY,PARTREPLACE,PARTCOVERED,PARTSHIP,DENIALCODE,PARTCONDITION,PARTCOMMENT,DATESUBMITTED,DATEUPDATED,DELETED,SHIPREQUESTED,SHIPORDERNO,PARTNOSHIPPED,SHIPFINISHED,SPIFF,PAYSPIFF,APEXPORTED,INVOICEID,TRACKING_1,TRACKING_2,SHIPMENT_DATE,SHIPMENT_CODE,ORDERDATE,PRESHIPPEDFLAG,REMAINUNIT,PARTSNUMBER,APPROVEDSPIFF,CREDIT,ISCREDITEXPORTED,LISTPRICE,SO_ID,COMPANY_ID,CLAIMSHIPMENTID,WEBCLAIMID,TRACKING_3,TRACKING_4,TRACKING_5,TRACKING_6,TRACKING_7,D2_VERSION,INVOICEID_SO_ITEM,SO_COMPANY_ITEM
0,1480228,1225821,1,RCX97400,MOTOR ASSY - SV/EV,1.00,True,True,True,None,None,None,2024-07-28 14:42:12.547,NaT,False,False,None,None,False,40.00,True,False,0,None,None,NaT,NaN,NaT,None,None,None,40.00,0.00,False,663.24,0,0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Version_0,0_0_RCX97400,0_0.0_RCX97400
1,1480210,1225801,3,LMCUS11050,UCL COLORLOGIC 80 ACCENT LIGHT,1.00,True,True,True,None,None,None,2024-07-28 11:36:01.587,NaT,False,False,None,None,False,20.19,False,False,0,None,None,NaT,NaN,NaT,None,None,None,NaN,0.00,False,244.77,0,0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Version_0,0_0_LMCUS11050,0_0.0_LMCUS11050
2,1480199,1216739,1,SPX3206Z1VSPE,PWR END-TRISTAR VS,1.00,True,True,True,None,None,None,2024-07-28 10:35:39.767,NaT,False,False,None,None,False,40.00,True,False,0,None,None,NaT,NaN,NaT,None,None,None,40.00,0.00,False,"3,094.28",0,0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Version_0,0_0_SPX3206Z1VSPE,0_0.0_SPX3206Z1VSPE
3,1480247,1225836,1,LACUS11100,100' COLORLOGIC 320 ACENT LIGHT,1.00,True,True,True,None,None,None,2024-07-28 17:29:11.997,NaT,False,False,None,None,False,0.00,False,False,0,None,None,NaT,NaN,NaT,None,None,None,NaN,0.00,False,833.15,0,0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Version_0,0_0_LACUS11100,0_0.0_LACUS11100
4,1480241,1225828,1,FDXLGSV0001,GAS VALVE NAT. KIT FD,1.00,True,True,True,None,None,None,2024-07-28 16:04:32.733,NaT,False,False,None,None,False,34.89,True,False,0,None,None,NaT,NaN,NaT,None,None,None,34.89,0.00,False,422.88,0,0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Version_0,0_0_FDXLGSV0001,0_0.0_FDXLGSV0001


# Warranty Servicers

In [42]:
# warranty serviced
df_warranty_claims.head()

,CLAIMID,SERVICINGID,SERVICINGCOMPANY,SERVICINGADDRESS1,SERVICINGADDRESS2,SERVICINGCITY,SERVICINGSTATE,SERVICINGZIP,SERVICINGPHONE,SERVICINGFAX,SERVICINGEMAIL,REPAIRTYPE,CUSTOMERNAME,CUSTOMERADDRESS,CUSTOMERCITY,CUSTOMERSTATE,CUSTOMERZIP,CUSTOMERPHONE,INSTALLERID,INSTALLERNAME,INSTALLERADDRESS,INSTALLERCITY,INSTALLERSTATE,INSTALLERZIP,INSTALLERPHONE,INSTALLATIONDATE,FAILUREDATE,REPAIRDATE,POOLTYPE,PRODUCTTYPE,PRODUCTID,MODELNO,SERIALNO,MOTORSERIALNO,FAILURECODE,COMPONENTCODE,REPAIRCODE,PROBLEMREPORTED,PROBLEMFOUND,REPAIRSPERFORMED,R22REFRIGERANT,LABOR,SALESACCOMMODATION,SALESPERSON,HEATPUMPPAYMENT,DATESUBMITTED,DATEUPDATED,STATUS,DENIALCODE,COMMENTS,ADMINCOMMENTS,ADDITIONALLABOR,DENIALCODE_LABOR,CHANGESTATUSDATE,WEBCLAIMID,SERIAL_VALID,IMPORTEDDATE,FORMENTRYDATE,SUBMITTEDSERIALNO,POSSIBLESERIALNO1,POSSIBLESERIALNO2,SERIALILLEGIBLE,SERIALISCORRECT,OLDSERIAL,SERIALMISSING,FORMTYPE,UPDATE_USERID,UPDATE_DATE,PAPERCLAIMNO,LABOR_RATE,SPOC_ADJ,MISCPART_ADJ,TOTAL_LABOR,TOTAL_OTHER,TOTAL_SPIFF,LABOR_APEXPORTED,SPIFF_APEXPORTED,OTHER_APEXPORTED,BETA,AUTHORIZATIONNUM,TOTAL_TAX,TAX_APEXPORTED,SALESPERSONID,AVSHIP,INSTALLEDBYPROF,ONLINEDEALER,AUDIT_SHA2_PK,AUDIT_TIMESTAMP
0,623469,119994-99,Big State Pools,13710 HUFFMEISTER RD,,Cypress,TX,77429,281-256-7886,281-256-7886,rharling@sbcglobal.net,1,Ruch Myers,15234 BEISERT CIR,MAGNOLIA,TX,77355-4210,281-330-9155,108731,Mckenz.net,16450 LONE STAR RD,NAVASOTA,TX,77868-8341,713-254-6608,2015-08-31,2017-07-07,2017-07-11,I,7,228,AQR,3A11061210976,,ES,CB,,AQR isn't working,bad board,Replaced board,0.00,0.00,False,,False,2017-07-12 10:51:18.570,NaT,1,,,,1.00,,2017-07-12 15:06:42,W623469,True,NaT,2017-07-12 10:51:06,None,None,None,False,False,False,False,1,29,2017-07-12 10:51:18.570,,90.00,0.00,NaN,90.00,0.00,33.11,90.00,33.11,0.00,False,,NaN,NaN,0.00,False,0.00,False,"b""\x16e46\xdb\x0c1)\xfe\x1bK\xd5\x14*L\x06zh\x...",2024-01-16 09:53:39.015000-05:00
1,623470,124912-99,Marschand / Etheridge Sales,5000 County Road 564 B,,Bushnell,FL,33513,352-793-2004,352-793-9771,larrymarschand@wildblue.net,1,BILL ROWE,1774 SOARING HEIGHTS CIR,ORLANDO,FL,32837-8079,407-856-1716,124912-99,Marschand / Etheridge Sales,5000 County Road 564 B,Bushnell,FL,33513,352-793-2004,2013-09-17,2017-06-21,2017-06-23,I,2,363,SP3400VSP,21111305051191003,,CK,PH,,THE PUMP IS SPRAYING WATER,THE HOUSING IS CRACKED,REPLACED THE HOUSING,0.00,0.00,False,,False,2017-07-12 10:51:41.107,NaT,1,,,,1.00,,2017-07-17 12:20:22,W623470,True,NaT,2017-07-12 10:47:56,None,None,None,False,False,False,False,1,29,2017-07-12 10:51:41.107,,75.00,0.00,NaN,75.00,0.00,0.00,75.00,0.00,0.00,False,479285,NaN,NaN,0.00,False,1.00,False,b'\x13!7l\xa1\xe4\x18\xcal\x1c\xe7c\xa0\x08\x1...,2024-01-16 09:53:39.015000-05:00
2,623471,104684,Star-Lite Pools,"10875 NW 52nd St, Ste 8",,Sunrise,FL,33351,954-747-3377,954-747-1494,starlitepoolbldrs@hotmail.com,1,Gillman,300 E KEY PALM RD,BOCA RATON,FL,33432-7925,561-395-2150,104684,Star-Lite Pools,10875 NW 52ND ST STE 8,SUNRISE,FL,33351-8086,954-747-3377,2016-09-15,2017-06-30,2017-07-11,I,6,774,LACUS11150,1u161031001212,,PW,BU,,light out,light out,replaced complete,0.00,0.00,False,,False,2017-07-12 10:56:21.570,NaT,1,,6 of 7,,1.00,,2017-07-17 12:22:11,W623471,True,NaT,2017-07-12 10:52:10,None,None,None,False,False,False,False,1,29,2017-07-12 10:56:21.570,,75.00,0.00,NaN,75.00,0.00,0.00,75.00,0.00,0.00,False,,NaN,NaN,0.00,False,0.00,False,b'6\xf0\xa6\x95X\xd5\x97z\xbc\xc4?7\xc0\xddS`l...,2024-01-16 09:53:39.015000-05:00
3,623472,139310-99,Cardillo Pools & Spas,56 Carriage Hill Rd,,Brewster,NY,10509,914-235-0883,914-235-0885,info@cardillopools.com,1,Gettinger,50 SHELDRAKE RD,SCARSDALE,NY,10583-3410,914-235-0883,139310-99,Cardillo Pools & Spas,56 CARRIAGE HILL RD,BREWSTER,NY,10509-3427,914-235-0883,2014-06-20,2017-07-10,2017-07-11,I,3,17,DE6020,21111409050850002,,CK,MA,,Pool water cloudy,Manifold was cracked,Open the filter and replaced the manifolds,0.00,0.00,False,,False,2017-07-12 11:02:43.037,NaT,5,,,Model DE6020 manufactured 09

In [43]:
df_warranty_claims.shape

(1099991, 88)

In [44]:
# filter approved claims
df_warranty_claims_approved = df_warranty_claims[df_warranty_claims['STATUS'].isin([1,5]) ]
df_warranty_claims_approved.head()

,CLAIMID,SERVICINGID,SERVICINGCOMPANY,SERVICINGADDRESS1,SERVICINGADDRESS2,SERVICINGCITY,SERVICINGSTATE,SERVICINGZIP,SERVICINGPHONE,SERVICINGFAX,SERVICINGEMAIL,REPAIRTYPE,CUSTOMERNAME,CUSTOMERADDRESS,CUSTOMERCITY,CUSTOMERSTATE,CUSTOMERZIP,CUSTOMERPHONE,INSTALLERID,INSTALLERNAME,INSTALLERADDRESS,INSTALLERCITY,INSTALLERSTATE,INSTALLERZIP,INSTALLERPHONE,INSTALLATIONDATE,FAILUREDATE,REPAIRDATE,POOLTYPE,PRODUCTTYPE,PRODUCTID,MODELNO,SERIALNO,MOTORSERIALNO,FAILURECODE,COMPONENTCODE,REPAIRCODE,PROBLEMREPORTED,PROBLEMFOUND,REPAIRSPERFORMED,R22REFRIGERANT,LABOR,SALESACCOMMODATION,SALESPERSON,HEATPUMPPAYMENT,DATESUBMITTED,DATEUPDATED,STATUS,DENIALCODE,COMMENTS,ADMINCOMMENTS,ADDITIONALLABOR,DENIALCODE_LABOR,CHANGESTATUSDATE,WEBCLAIMID,SERIAL_VALID,IMPORTEDDATE,FORMENTRYDATE,SUBMITTEDSERIALNO,POSSIBLESERIALNO1,POSSIBLESERIALNO2,SERIALILLEGIBLE,SERIALISCORRECT,OLDSERIAL,SERIALMISSING,FORMTYPE,UPDATE_USERID,UPDATE_DATE,PAPERCLAIMNO,LABOR_RATE,SPOC_ADJ,MISCPART_ADJ,TOTAL_LABOR,TOTAL_OTHER,TOTAL_SPIFF,LABOR_APEXPORTED,SPIFF_APEXPORTED,OTHER_APEXPORTED,BETA,AUTHORIZATIONNUM,TOTAL_TAX,TAX_APEXPORTED,SALESPERSONID,AVSHIP,INSTALLEDBYPROF,ONLINEDEALER,AUDIT_SHA2_PK,AUDIT_TIMESTAMP
0,623469,119994-99,Big State Pools,13710 HUFFMEISTER RD,,Cypress,TX,77429,281-256-7886,281-256-7886,rharling@sbcglobal.net,1,Ruch Myers,15234 BEISERT CIR,MAGNOLIA,TX,77355-4210,281-330-9155,108731,Mckenz.net,16450 LONE STAR RD,NAVASOTA,TX,77868-8341,713-254-6608,2015-08-31,2017-07-07,2017-07-11,I,7,228,AQR,3A11061210976,,ES,CB,,AQR isn't working,bad board,Replaced board,0.00,0.00,False,,False,2017-07-12 10:51:18.570,NaT,1,,,,1.00,,2017-07-12 15:06:42,W623469,True,NaT,2017-07-12 10:51:06,None,None,None,False,False,False,False,1,29,2017-07-12 10:51:18.570,,90.00,0.00,NaN,90.00,0.00,33.11,90.00,33.11,0.00,False,,NaN,NaN,0.00,False,0.00,False,"b""\x16e46\xdb\x0c1)\xfe\x1bK\xd5\x14*L\x06zh\x...",2024-01-16 09:53:39.015000-05:00
1,623470,124912-99,Marschand / Etheridge Sales,5000 County Road 564 B,,Bushnell,FL,33513,352-793-2004,352-793-9771,larrymarschand@wildblue.net,1,BILL ROWE,1774 SOARING HEIGHTS CIR,ORLANDO,FL,32837-8079,407-856-1716,124912-99,Marschand / Etheridge Sales,5000 County Road 564 B,Bushnell,FL,33513,352-793-2004,2013-09-17,2017-06-21,2017-06-23,I,2,363,SP3400VSP,21111305051191003,,CK,PH,,THE PUMP IS SPRAYING WATER,THE HOUSING IS CRACKED,REPLACED THE HOUSING,0.00,0.00,False,,False,2017-07-12 10:51:41.107,NaT,1,,,,1.00,,2017-07-17 12:20:22,W623470,True,NaT,2017-07-12 10:47:56,None,None,None,False,False,False,False,1,29,2017-07-12 10:51:41.107,,75.00,0.00,NaN,75.00,0.00,0.00,75.00,0.00,0.00,False,479285,NaN,NaN,0.00,False,1.00,False,b'\x13!7l\xa1\xe4\x18\xcal\x1c\xe7c\xa0\x08\x1...,2024-01-16 09:53:39.015000-05:00
2,623471,104684,Star-Lite Pools,"10875 NW 52nd St, Ste 8",,Sunrise,FL,33351,954-747-3377,954-747-1494,starlitepoolbldrs@hotmail.com,1,Gillman,300 E KEY PALM RD,BOCA RATON,FL,33432-7925,561-395-2150,104684,Star-Lite Pools,10875 NW 52ND ST STE 8,SUNRISE,FL,33351-8086,954-747-3377,2016-09-15,2017-06-30,2017-07-11,I,6,774,LACUS11150,1u161031001212,,PW,BU,,light out,light out,replaced complete,0.00,0.00,False,,False,2017-07-12 10:56:21.570,NaT,1,,6 of 7,,1.00,,2017-07-17 12:22:11,W623471,True,NaT,2017-07-12 10:52:10,None,None,None,False,False,False,False,1,29,2017-07-12 10:56:21.570,,75.00,0.00,NaN,75.00,0.00,0.00,75.00,0.00,0.00,False,,NaN,NaN,0.00,False,0.00,False,b'6\xf0\xa6\x95X\xd5\x97z\xbc\xc4?7\xc0\xddS`l...,2024-01-16 09:53:39.015000-05:00
3,623472,139310-99,Cardillo Pools & Spas,56 Carriage Hill Rd,,Brewster,NY,10509,914-235-0883,914-235-0885,info@cardillopools.com,1,Gettinger,50 SHELDRAKE RD,SCARSDALE,NY,10583-3410,914-235-0883,139310-99,Cardillo Pools & Spas,56 CARRIAGE HILL RD,BREWSTER,NY,10509-3427,914-235-0883,2014-06-20,2017-07-10,2017-07-11,I,3,17,DE6020,21111409050850002,,CK,MA,,Pool water cloudy,Manifold was cracked,Open the filter and replaced the manifolds,0.00,0.00,False,,False,2017-07-12 11:02:43.037,NaT,5,,,Model DE6020 manufactured 09

In [45]:
# free up memory
del df_warranty_claims

In [46]:
# rename columns so as not to conflict with columns of same name in df_warranty_claims_parts
df_warranty_claims_approved.rename(columns={"DATESUBMITTED":"DATESUBMITTED_FOR_SERVICING"},inplace=True)
df_warranty_claims_approved.rename(columns={"DENIALCODE":"DENIALCODE_FOR_SERVICING"},inplace=True)

In [47]:
# remove unnecessary columns
columns_to_drop = ['SERVICINGADDRESS1', 'SERVICINGADDRESS2', 'SERVICINGPHONE', 'SERVICINGFAX', 'SERVICINGEMAIL', 'CUSTOMERNAME',
                   'CUSTOMERADDRESS', 'CUSTOMERCITY', 'CUSTOMERSTATE', 'CUSTOMERZIP', 'CUSTOMERPHONE', 'INSTALLERADDRESS',
                  'INSTALLERZIP', 'R22REFRIGERANT', 'HEATPUMPPAYMENT', 'DATEUPDATED', 'WEBCLAIMID', 'IMPORTEDDATE', 
                   'FORMENTRYDATE', 'SUBMITTEDSERIALNO', 'POSSIBLESERIALNO1', 'POSSIBLESERIALNO2', 'SERIALILLEGIBLE', 
                   'SERIALISCORRECT', 'OLDSERIAL', 'SERIALMISSING', 'FORMTYPE', 'UPDATE_USERID', 'UPDATE_DATE','PAPERCLAIMNO', 
                   'INSTALLEDBYPROF', 'ONLINEDEALER', 'AUDIT_SHA2_PK', 'AUDIT_TIMESTAMP']
df_warranty_claims_approved.drop(columns=columns_to_drop, inplace=True)

In [48]:
# is claimid unique?
print('Total Unique Records:', df_warranty_claims_approved['CLAIMID'].nunique())
print('Total Length of Records:', len(df_warranty_claims_approved['CLAIMID']))
print('Is field unique:', df_warranty_claims_approved['CLAIMID'].is_unique)

Total Unique Records: 1051215
Total Length of Records: 1051215
Is field unique: True


In [49]:
df_warranty_claims_approved.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1051215 entries, 0 to 1099990
Data columns (total 54 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   CLAIMID                      1051215 non-null  int32         
 1   SERVICINGID                  1051215 non-null  object        
 2   SERVICINGCOMPANY             1051215 non-null  object        
 3   SERVICINGCITY                1051215 non-null  object        
 4   SERVICINGSTATE               1051215 non-null  object        
 5   SERVICINGZIP                 1051215 non-null  object        
 6   REPAIRTYPE                   1051215 non-null  int8          
 7   INSTALLERID                  1051215 non-null  object        
 8   INSTALLERNAME                1051215 non-null  object        
 9   INSTALLERCITY                1051215 non-null  object        
 10  INSTALLERSTATE               1051215 non-null  object        
 11  INSTALLERPHONE  

In [50]:
# join test
# df_merge_test_1 = pd.merge(df_warranty_claims_parts_all, df_warranty_claims_approved, 
#                            on='CLAIMID', how='left')
# df_merge_test_1.head()

In [51]:
# join test successful, save memory
# del df_merge_test_1

# Serial Numbers

In [26]:
df_serial.shape

(53030730, 11)

In [8]:
# remove unnecessary columns
columns_to_drop = ['PARTNO', 'CLASSID', 'FMLYID', 'MARKETID', 'PK', 'AUDIT_SHA2_PK', 'AUDIT_TIMESTAMP']
df_serial.drop(columns=columns_to_drop, inplace=True)

In [27]:
# serial numbers
df_serial.head()

,SERIALNO,DATEMFG,MFGID,PARTNO,CLASSID,FMLYID,BUSID,MARKETID,PK,AUDIT_SHA2_PK,AUDIT_TIMESTAMP
0,21120707001390018,2007-07-03,4,SP0714TX,110,4Q,5,3,3971220,b'\xd0\xc7>i\x81l5 \xbc\xc0\xde\xa8\x1fz&q\xc8...,2024-01-16 09:44:04.696000-05:00
1,21120707001390019,2007-07-03,4,SP0714TX,110,4Q,5,3,3971221,b'\xaa\xe7\xab$\xd1\x19\xd7\xa5\xae\x07ro\xd4T...,2024-01-16 09:44:04.696000-05:00
2,21120707001390020,2007-07-03,4,SP0714TX,110,4Q,5,3,3971222,"b'\xbb\x1bt~Z\xae\xd8\xf2,[\xb8\x7f\x8f\x1f\x1...",2024-01-16 09:44:04.696000-05:00
3,21120707001390021,2007-07-03,4,SP0714TX,110,4Q,5,3,3971223,b'\xdf8}r\x9c\xa5\xc0\x9e\xcd6\xc1\xff\x9e\xd9...,2024-01-16 09:44:04.696000-05:00
4,21120707001390022,2007-07-03,4,SP0714TX,110,4Q,5,3,3971224,b'\x8b\xe6\xff\x99\xbeX\xc7\xa2i\x82\xbd\xd39\...,2024-01-16 09:44:04.696000-05:00


In [19]:
#delete later
df_serial_2022 = df_serial[df_serial['DATEMFG'].astype(str).str.contains('2022')]
df_serial_2022.tail()

,SERIALNO,DATEMFG,MFGID,PARTNO,CLASSID,FMLYID,BUSID,MARKETID,PK,AUDIT_SHA2_PK,AUDIT_TIMESTAMP
52501622,3E22089714201,2022-03-30,11,T-CELL-3,870,5M,9,3,53015089,b'\x9drP\x08\xd8\xcc\xbf\xc6\x0b\x02S\x90\x07\...,2024-07-10 03:41:26.003000-04:00
52727560,21111910050102017,2022-12-16,11,SP303053,101,1A,4,3,34459187,b'\x1e\xc0\xcc\x1d\xe2 \xc8GD\x0cJk\xf1\x9aX\x...,2024-01-16 09:44:04.696000-05:00
52727562,21111910050103001,2022-12-16,11,SP303053,101,1A,4,3,34459189,b'I\xb7\xcbn\xa6\x84\xac\x07<\xf2\xb7\xb2\xf0\...,2024-01-16 09:44:04.696000-05:00
52727563,21111910050103002,2022-12-16,11,SP303053,101,1A,4,3,34459190,"b'O|l5`w\xe4\x00\x948;\x8dj\xf4k\x89)\xa6\x83""...",2024-01-16 09:44:04.696000-05:00
52727567,21111910050103006,2022-12-16,11,SP303053,101,1A,4,3,34459194,"b""\rD\x86\x84\x91\x02I\xf3\x80M\xcc`\x1c\x00By...",2024-01-16 09:44:04.696000-05:00


In [31]:
df_serial[df_serial['SERIALNO']=='21122202007056013']

,SERIALNO,DATEMFG,MFGID,PARTNO,CLASSID,FMLYID,BUSID,MARKETID,PK,AUDIT_SHA2_PK,AUDIT_TIMESTAMP
14101487,21122202007056013,2022-02-19,4,SP10852OM,150,4B,5,3,46184574,b'\x0e1\xdbS\xd5\x11\xa0Z;\x8c$\xa1\xab\x99\x1...,2024-01-16 09:44:04.696000-05:00


In [10]:
# if csv is needed
df_serial.to_csv('serial_numbers.csv')

In [55]:
df_serial['MFG_MONTH'] = df_serial['DATEMFG'].dt.month.astype('Int16')
df_serial['MFG_YEAR'] = df_serial['DATEMFG'].dt.year.astype('Int16')
df_serial['MFG_YEAR_MONTH'] = df_serial['MFG_YEAR'].astype(str) + '_' + df_serial['MFG_MONTH'].astype(str).str.zfill(2)

In [28]:
# is serial unique?
print('Total Unique Records:', df_serial['SERIALNO'].nunique())
print('Total Length of Records:', len(df_serial['SERIALNO']))
print('Is field unique:', df_serial['SERIALNO'].is_unique)

Total Unique Records: 53030730
Total Length of Records: 53030730
Is field unique: True


In [57]:
# join test
# df_merge_test_2 = pd.merge(df_warranty_claims_approved, df_serial, on='SERIALNO', how='left')
# df_merge_test_2.head()

In [58]:
# join test successful, save memory
# del df_merge_test_2

# Invoices

In [59]:
# invoices
df_invoice.head()

,PARENT_COMPANY_ID,COMPANY_ID,BILL_CUST_NBR,BILL_CUST_NBR_SEQ,SHIP_CUST_NBR,SHIP_CUST_NBR_SEQ,INV_NUM,INV_DATE,INV_CALENDAR_KEY,SO_ID,SO_LINE_NUM,INV_LINE_ITEM_ID,ITEM_ID,CUSTOMER_PARENT_KEY,INV_PARENT_COMPANY_ID,INV_SHIPMENT_SEQ,INV_SHIP_TO_ZIP,INV_SHIP_TO_CITY,INV_SHIP_TO_PROVINCE_CD,INV_SHIP_TO_PROVINCE_DESC,INV_SHIP_TO_STATE_CD,INV_SHIP_TO_STATE_DESC,INV_COUNTRY_CD,INV_COUNTRY_DESC,INV_CONTINENTAL_REGION_CD,INV_CONTINENTAL_REGION_DESC,ACTUAL_SHIPMENT_DATE,SO_PARENT_OF_CUST_ACCT_SHIP_TO_ID,INV_CUST_ACCT_SHIP_TO_ID,INV_CUST_ACCT_BILL_TO_ID,INV_SHIPMENT_METHOD_CD,INV_SHIP_TO_ACCT_NAME,INV_SHIP_TO_TERRITORY_CD,INV_SHIP_TO_TERRITORY_DESC,INV_TOTAL_NON_TAXABLE_AMT,INV_TOTAL_INVOICED_NET_AMT,INV_TOTAL_INVOICED_ITEMS_QTY,INV_TOTAL_INVOICED_ITEMS_AMT,INV_TERMS_CD,INV_TERMS_DESC,INV_TOTAL_TAX_AMT,INV_TOTAL_TAXABLE_AMT,INV_LINE_PRIOR_SHIPPED_QTY,INV_LINE_INVOICED_QTY,INV_LINE_AMT_COMP_CURRENCY,INV_LINE_AMT_USD,SOURCE_SYSTEM_FLAG,INV_ORDER_TYPE_CLASSIFICATION_CD,SHIP_TO_CUSTOMER_PARENT_KEY,BILL_TO_CUSTOMER_PARENT_KEY
0,1,60,137550,0.00,69380609,0,600333929,2024-06-26,20240626,401859,1.00,005402383400,005402383400,1-69380609,1,1,77007,HOUSTON,None,None,TX,TEXAS,USA,United States of America,None,United States,2024-06-26,69380609,69380609-000000,137550-000000,201,AVEA POOLS,24,None,0.00,0.00,1.00,0.00,004,None,0.00,0.00,0.00,1,0.00,0.00,USA,SOM,1-69380609,1-137550
1,1,1,437500,0.00,437500,0,13138289,2024-06-26,20240626,134401,28.00,SMX303140003,SMX303140003,1-437500,1,6,L6H 5R4,OAKVILLE ONT,ON,ONTARIO,FO,FOREIGN,CAN,Canada,None,Canada,2024-06-26,437500,437500-000000,437500-000000,371,HAYWARD POOL PRODS CANADA INC,None,None,0.00,196.20,5.00,196.20,015,NET 60 DAYS,0.00,0.00,2.00,2,85.68,85.68,USA,SOM,1-437500,1-437500
2,1,1,437500,0.00,437500,0,13138289,2024-06-26,20240626,134401,27.00,SMX303140002,SMX303140002,1-437500,1,6,L6H 5R4,OAKVILLE ONT,ON,ONTARIO,FO,FOREIGN,CAN,Canada,None,Canada,2024-06-26,437500,437500-000000,437500-000000,371,HAYWARD POOL PRODS CANADA INC,None,None,0.00,196.20,5.00,196.20,015,NET 60 DAYS,0.00,0.00,0.00,3,110.52,110.52,USA,SOM,1-437500,1-437500
3,1,9,118805,495.00,118805,495,90732685,2024-06-26,20240626,618768,1.00,21101,21101,1-118805,1,1,75248,DALLAS,None,None,TX,TEXAS,USA,United States of America,None,United States,2024-06-26,118805,118805-000495,118805-000495,200,POOLWERX HIGHLAND PARK,14,None,0.00,"2,168.05",5.00,"2,168.05",001,None,0.00,0.00,0.00,5,"2,168.05","2,168.05",USA,SOM,1-118805,1-118805
4,1,60,175067,85.00,175067,85,600334001,2024-06-26,20240626,403504,14.00,SX310DA2,SX310DA2,1-175067,1,8,85233,GILBERT,None,None,AZ,ARIZONA,USA,United States of America,None,United States,2024-06-26,175067,175067-000085,175067-000085,201,"SCP DISTRIBUTORS, LLC",24,None,0.00,374.96,7.00,374.96,001,None,0.00,0.00,0.00,3,278.91,278.91,USA,SOM,1-175067,1-175067


In [60]:
# change data type
# convert to numeric, coercing errors to NaN, then fill NaN with 0
df_invoice['INV_NUM'] = pd.to_numeric(df_invoice['INV_NUM'], errors='coerce').fillna(0).astype('Int32')

In [61]:
# remove unnecessary columns
columns_to_drop = ['PARENT_COMPANY_ID', 'BILL_CUST_NBR', 'BILL_CUST_NBR_SEQ', 'SHIP_CUST_NBR', 
                   'SHIP_CUST_NBR', 'INV_CALENDAR_KEY', 'CUSTOMER_PARENT_KEY', 'INV_PARENT_COMPANY_ID', 
                   'INV_SHIPMENT_SEQ', 'INV_SHIP_TO_ZIP', 'INV_SHIP_TO_CITY', 'INV_SHIP_TO_PROVINCE_CD', 
                   'INV_SHIP_TO_PROVINCE_DESC', 'INV_SHIP_TO_STATE_DESC', 'INV_COUNTRY_DESC', 'INV_CONTINENTAL_REGION_CD',
                   'INV_CONTINENTAL_REGION_DESC','SO_PARENT_OF_CUST_ACCT_SHIP_TO_ID','INV_CUST_ACCT_SHIP_TO_ID',
                   'INV_CUST_ACCT_BILL_TO_ID','INV_SHIPMENT_METHOD_CD', 'INV_SHIP_TO_TERRITORY_CD',
                   'INV_SHIP_TO_TERRITORY_DESC','INV_TOTAL_NON_TAXABLE_AMT','INV_TOTAL_INVOICED_NET_AMT',
                   'INV_TOTAL_INVOICED_ITEMS_QTY', 'INV_TOTAL_INVOICED_ITEMS_AMT', 'INV_TERMS_CD','INV_TERMS_DESC',
                   'INV_TOTAL_TAX_AMT','INV_TOTAL_TAXABLE_AMT','INV_LINE_PRIOR_SHIPPED_QTY',
                   'INV_LINE_AMT_COMP_CURRENCY','SOURCE_SYSTEM_FLAG','INV_ORDER_TYPE_CLASSIFICATION_CD',
                   'SHIP_TO_CUSTOMER_PARENT_KEY','BILL_TO_CUSTOMER_PARENT_KEY', ]

df_invoice.drop(columns=columns_to_drop, inplace=True)

In [62]:
# rename company_id infoice, so_id_invoice, SO_LINE_NUM
# rename columns so as not to conflict with columnx of same name in df_warranty_claims_parts
df_invoice.rename(columns={"COMPANY_ID":"COMPANY_ID_FOR_INVOICE"},inplace=True)
df_invoice.rename(columns={"SO_ID":"SO_ID_FOR_INVOICE"},inplace=True)
df_invoice.rename(columns={"SO_LINE_NUM":"SO_LINE_NUM_FOR_INVOICE"},inplace=True)

In [63]:
df_invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5222283 entries, 0 to 5222282
Data columns (total 14 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   COMPANY_ID_FOR_INVOICE   int16  
 1   SHIP_CUST_NBR_SEQ        int16  
 2   INV_NUM                  Int32  
 3   INV_DATE                 object 
 4   SO_ID_FOR_INVOICE        object 
 5   SO_LINE_NUM_FOR_INVOICE  float64
 6   INV_LINE_ITEM_ID         object 
 7   ITEM_ID                  object 
 8   INV_SHIP_TO_STATE_CD     object 
 9   INV_COUNTRY_CD           object 
 10  ACTUAL_SHIPMENT_DATE     object 
 11  INV_SHIP_TO_ACCT_NAME    object 
 12  INV_LINE_INVOICED_QTY    int32  
 13  INV_LINE_AMT_USD         float64
dtypes: Int32(1), float64(2), int16(2), int32(1), object(8)
memory usage: 463.2+ MB


In [64]:
# change data type to datetime
df_invoice['INV_DATE']=pd.to_datetime(df_invoice['INV_DATE'])

In [65]:
# is 'INV_NUM' unique?
print('Total Unique Records:', df_invoice['INV_NUM'].nunique())
print('Total Length of Records:', len(df_invoice['INV_NUM']))
print('Is field unique:', df_invoice['INV_NUM'].is_unique)

Total Unique Records: 2324083
Total Length of Records: 5222283
Is field unique: False


In [66]:
# INV_NUM is not a unique identifier but we can aggregate to make it unique
df_invoice_aggregate = df_invoice.groupby(['INV_NUM', 'COMPANY_ID_FOR_INVOICE', 'SO_ID_FOR_INVOICE', 
                                           'INV_LINE_ITEM_ID', 'INV_SHIP_TO_STATE_CD', 'INV_COUNTRY_CD', 
                                           'ACTUAL_SHIPMENT_DATE', 'INV_SHIP_TO_ACCT_NAME',  ]).agg({
                                           'INV_LINE_INVOICED_QTY': 'sum',
                                           'INV_LINE_AMT_USD': 'mean'
                                            }).reset_index()
df_invoice_aggregate.head()

,INV_NUM,COMPANY_ID_FOR_INVOICE,SO_ID_FOR_INVOICE,INV_LINE_ITEM_ID,INV_SHIP_TO_STATE_CD,INV_COUNTRY_CD,ACTUAL_SHIPMENT_DATE,INV_SHIP_TO_ACCT_NAME,INV_LINE_INVOICED_QTY,INV_LINE_AMT_USD
0,11980271,1,914846,SPX3400DRKIT,TX,USA,2014-10-08,MIKE EAKER,1,0.00
1,11995523,1,929422,LITCRNG15,IL,USA,2015-01-09,SCP DISTRIBUTORS,50,0.00
2,12047507,1,979096,WGX1033BLV2,PA,USA,2015-07-24,PLASTIC DEVELOPMENT CO,1,0.00
3,12190476,1,214865,LKCUS1100,GA,USA,2017-01-24,HAL DYE,6,0.00
4,12191743,1,215976,6060,PA,USA,2017-01-30,AQUA-MATIC POOLS LLC,1,0.00


In [67]:
# create a key to use with df_invoice_aggregate so there is a unique key
df_invoice_aggregate['INVOICEID_SO_ITEM'] = ( df_invoice_aggregate['INV_NUM'].astype(str) 
                              + '_' 
                              + df_invoice_aggregate['SO_ID_FOR_INVOICE'].astype(str)
                              + '_' 
                              + df_invoice_aggregate['INV_LINE_ITEM_ID'].fillna(0).astype(str) )

# confirm key works
df_invoice_aggregate['INVOICEID_SO_ITEM']

0          11980271_914846_SPX3400DRKIT
1             11995523_929422_LITCRNG15
2           12047507_979096_WGX1033BLV2
3             12190476_214865_LKCUS1100
4                  12191743_215976_6060
                       ...             
4150576          600342473_414490_27516
4150577         600342473_414490_997017
4150578          600342473_414490_99960
4150579          600342473_414490_99961
4150580          600342473_414490_99980
Name: INVOICEID_SO_ITEM, Length: 4150581, dtype: object

In [68]:
# is 'INVOICEID_SO_ITEM' unique for aggregate table?
print('Total Unique Records:', df_invoice_aggregate['INVOICEID_SO_ITEM'].nunique())
print('Total Length of Records:', len(df_invoice_aggregate['INVOICEID_SO_ITEM']))
print('Is field unique:', df_invoice_aggregate['INVOICEID_SO_ITEM'].is_unique)

Total Unique Records: 4150581
Total Length of Records: 4150581
Is field unique: True


In [69]:
# join test
# df_merge_test_3 = pd.merge(df_warranty_claims_parts_all, df_invoice_aggregate,
#                            on='INVOICEID_SO_ITEM', how='left')

# # normal for new claims not to have an invoice yet, so checking 2020 to confirm join
# df_merge_test_3[df_merge_test_3['DATESUBMITTED'] >= '1/1/2020'].head()

In [70]:
# check if invoiceid and inv_num are the same
# all_values_match = (df_merge_test_3['INV_NUM'] == df_merge_test_3['INVOICEID']).all()
# all_values_match

In [71]:
# # join test successful, save memory
# del df_merge_test_3

# Accounts Payable

In [72]:
# ap data
df_ap.head()

,AP_PAYMENTID,WEBCLAIMID,PAYMENTDATE,PROCESSSTATUS,SPIFF_PAID,LABOR_PAID,MISC_PAID,APVOUCHER,BATCHID,FMLYID,GLWARRANTY,GLLABOR,PAYMENTNO,PAYMENTIMPORTDATE,APID,PARENTCONO,HST_TAX,QST_TAX,GST_TAX,TAX_PAID,GL_GST,GL_HST,GL_QST,AUDIT_SHA2_PK,AUDIT_TIMESTAMP
0,684722,W944211,2021-12-10,C,8.48,75.00,0.00,902623290,APWARR211209,5S,2731115531116,2731115531113,000003008468,2021-12-11 01:00:17.617,0150477,001,0.00,0.00,0.00,0.00,None,None,None,b'\xec~\xc1:a\xeb\x1bOH\xc3\x94\xf5\x04W0.\x8d...,2024-01-16 09:56:29.608000-05:00
1,684723,W944686,2021-12-10,C,40.00,0.00,0.00,902623587,APWARR211209,5M,2731115531116,2731115531113,000003008582,2021-12-11 01:00:17.617,0151104,001,0.00,0.00,0.00,0.00,None,None,None,"b',\x01X&\xc4\xd3cq\xc5[\x1b\xf3(\x91rB\x9c\x1...",2024-01-16 09:56:29.608000-05:00
2,684724,W943978,2021-12-10,C,0.00,60.00,0.00,902623203,APWARR211209,1Y,2131115531116,2131115531113,000003008464,2021-12-11 01:00:17.617,0151205,001,0.00,0.00,0.00,0.00,None,None,None,b'aj\x0b\x90\xa8\x0b\xd1\x9d\x80\x8a\x85\xd4?\...,2024-01-16 09:56:29.608000-05:00
3,684725,W944237,2021-12-10,C,40.00,100.00,0.00,902623308,APWARR211209,5M,2731115531116,2731115531113,000003008467,2021-12-11 01:00:17.617,0151881B,001,0.00,0.00,0.00,0.00,None,None,None,b'\xe2\xcb\xd2O\xf0\x0f\xa3U5\xfaX-\xa2>\x12\x...,2024-01-16 09:56:29.608000-05:00
4,684726,W944239,2021-12-10,C,15.71,100.00,0.00,902623309,APWARR211209,3B,2131115531116,2131115531113,000003008467,2021-12-11 01:00:17.617,0151881B,001,0.00,0.00,0.00,0.00,None,None,None,b'\xb1\x95\xc5\xd7Y\x13\xe9\xd9\xce1\xc3\xae\x...,2024-01-16 09:56:29.608000-05:00


In [73]:
# is 'WEBCLAIMID' unique?
print('Total Unique Records:', df_ap['WEBCLAIMID'].nunique())
print('Total Length of Records:', len(df_ap['WEBCLAIMID']))
print('Is field unique:', df_ap['WEBCLAIMID'].is_unique)

Total Unique Records: 819599
Total Length of Records: 835389
Is field unique: False


In [74]:
# let's see what the duplicates (non unique values) look like
df_ap_duplicates = df_ap[df_ap.duplicated(
                        subset='WEBCLAIMID', keep=False)].sort_values('WEBCLAIMID', ascending=True)
df_ap_duplicates.sort_values('WEBCLAIMID', ascending=True).head(5)

,AP_PAYMENTID,WEBCLAIMID,PAYMENTDATE,PROCESSSTATUS,SPIFF_PAID,LABOR_PAID,MISC_PAID,APVOUCHER,BATCHID,FMLYID,GLWARRANTY,GLLABOR,PAYMENTNO,PAYMENTIMPORTDATE,APID,PARENTCONO,HST_TAX,QST_TAX,GST_TAX,TAX_PAID,GL_GST,GL_HST,GL_QST,AUDIT_SHA2_PK,AUDIT_TIMESTAMP
744481,39618,P155488,2009-06-23,C,0.00,0.00,-3.00,900441961,APWARR0906XY,3P,2131115531116,2131115531113,000000433042,2009-07-15 12:13:09.617,0164553,001,0.00,0.00,0.00,0.00,None,None,None,"b""S\xbe\x8c\xf2@\x9e\xe7\x94>\xa1\xd6f\xf8'-fe...",2024-01-16 09:56:29.608000-05:00
720923,3121,P155488,2009-02-25,C,0.00,0.00,3.00,900420201,APWARR090225,3P,2131115531114,2131115531113,000000423076,2009-03-25 11:08:37.403,None,001,0.00,0.00,0.00,0.00,None,None,None,b'7\xaf?g\x8e\xcfC\x84\x91\xe3B\xee1\x99j{)A\x...,2024-01-16 09:56:29.608000-05:00
718240,377,P155488,2009-02-06,C,0.00,93.75,3.00,900417187,APWARR090206,3P,2131115531114,2131115531113,000000421629,2009-03-25 11:04:32.750,None,001,0.00,0.00,0.00,0.00,None,None,None,"b',L\xf6W3x5\x12[\xc4%\x8d\x0e.Tj\xf4\x18[\xdb...",2024-01-16 09:56:29.608000-05:00
720924,3122,P155492,2009-02-25,C,0.00,0.00,27.00,900420201,APWARR090225,3P,2131115531114,2131115531113,000000423076,2009-03-25 11:08:37.420,None,001,0.00,0.00,0.00,0.00,None,None,None,b'\xc9\xb9\xc4\xc56\xccW\x87\xee!6]\xd9%\xa5\x...,2024-01-16 09:56:29.608000-05:00
744482,39619,P155492,2009-06-23,C,0.00,0.00,-27.00,900441961,APWARR0906XY,3P,2131115531116,2131115531113,000000433042,2009-07-15 12:13:09.617,0164553,001,0.00,0.00,0.00,0.00,None,None,None,b'\x0f\xc0\x1d\xa0x\xdc\xd5\x16B\x10\xa4\xc4MU...,2024-01-16 09:56:29.608000-05:00


In [75]:
# df_ap.WEBCLAIMID is not a unique identifier but we can aggregate to make it unique
df_ap_aggregate = df_ap.groupby(['WEBCLAIMID']).agg({
                'SPIFF_PAID': 'sum',
                'LABOR_PAID': 'sum',
                'MISC_PAID': 'sum'
}).reset_index()
df_ap_aggregate.head()

,WEBCLAIMID,SPIFF_PAID,LABOR_PAID,MISC_PAID
0,P130568,2.31,0.00,0.00
1,P155078,0.00,15.00,0.00
2,P155388,0.00,135.00,16.00
3,P155393,0.00,337.50,56.00
4,P155395,0.00,135.00,0.00


In [76]:
# free up memory
del df_ap
del df_ap_duplicates

In [77]:
# is 'WEBCLAIMID' unique in the aggregate?
print('Total Unique Records:', df_ap_aggregate['WEBCLAIMID'].nunique())
print('Total Length of Records:', len(df_ap_aggregate['WEBCLAIMID']))
print('Is field unique:', df_ap_aggregate['WEBCLAIMID'].is_unique)

Total Unique Records: 819599
Total Length of Records: 819599
Is field unique: True


In [78]:
# join test
# df_merge_test_4 = pd.merge(df_warranty_claims_parts_all, df_ap_aggregate, on='WEBCLAIMID', how='left')
# df_merge_test_4.head()

In [79]:
# join test successful, save memory
# del df_merge_test_4

# Distributor Credits

In [80]:
# distributor credits data
df_distributor_credits.head()

,COMPANY_ID,INVOICE_ID,SALES_ORDER_ID,PRODUCT_KEY,CUSTOMER_SHIP_TO_KEY,INVOICE_DATE,ITEM_STANDARD_COST_USD,ITEM_STANDARD_COST_LC,SHIP_TO_CITY,SHIP_TO_STATE,SHIP_TO_COUNTRY,SHIP_TO_CONTINENT,INVOICE_QTY,INVOICED_AMOUNT_LC,INVOICED_AMOUNT_USD
0,1,19202309,505528,1-S180T,1-175067-000105,2019-01-10,68.78,68.78,PLAINS,PENNSYLVANIA,United States of America,United States,1,211.75,211.75
1,1,19240427,760766,1-SP1580X15,1-175067-000007,2021-06-18,62.29,62.29,NASHVILLE,TENNESSEE,United States of America,United States,1,203.06,203.06
2,1,19259448,868608,1-SP2670007X10,1-115600-000000,2022-09-15,203.19,203.19,SAGINAW,MICHIGAN,United States of America,United States,1,478.33,478.33
3,1,19240412,760785,1-SP0607U,1-179320-000000,2021-06-18,23.31,23.31,LOUISVILLE,KENTUCKY,United States of America,United States,1,79.46,79.46
4,1,19240411,760316,1-SP2670010X15,1-100009-000002,2021-06-18,125.22,125.22,PERRYSBURG,OHIO,United States of America,United States,1,429.18,429.18


In [81]:
df_distributor_credits.shape

(100799, 15)

In [82]:
columns_to_drop = ['SHIP_TO_CONTINENT', 'ITEM_STANDARD_COST_LC', 'INVOICED_AMOUNT_LC']

df_distributor_credits.drop(columns=columns_to_drop, inplace=True)

In [83]:
df_distributor_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100799 entries, 0 to 100798
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   COMPANY_ID              100799 non-null  int16  
 1   INVOICE_ID              100799 non-null  object 
 2   SALES_ORDER_ID          100799 non-null  object 
 3   PRODUCT_KEY             100799 non-null  object 
 4   CUSTOMER_SHIP_TO_KEY    100799 non-null  object 
 5   INVOICE_DATE            100799 non-null  object 
 6   ITEM_STANDARD_COST_USD  100799 non-null  float64
 7   SHIP_TO_CITY            100553 non-null  object 
 8   SHIP_TO_STATE           100799 non-null  object 
 9   SHIP_TO_COUNTRY         100799 non-null  object 
 10  INVOICE_QTY             100799 non-null  int32  
 11  INVOICED_AMOUNT_USD     100799 non-null  float64
dtypes: float64(2), int16(1), int32(1), object(8)
memory usage: 8.3+ MB


In [84]:
# format
df_distributor_credits['INVOICE_ID'] = df_distributor_credits['INVOICE_ID'].astype('Int32')
df_distributor_credits['SALES_ORDER_ID'] = df_distributor_credits['SALES_ORDER_ID'].astype('Int32')
df_distributor_credits['INVOICE_DATE'] = pd.to_datetime(df_distributor_credits['INVOICE_DATE'], errors='coerce')

# make sure this is formatted exactly to ensure merging works as intended
df_distributor_credits['PRODUCT_KEY'] = df_distributor_credits['PRODUCT_KEY'].str.strip().str.upper().astype(str)

In [85]:
# add column to identify distributor credits vs. d2 data
df_distributor_credits['WARRANTY_SOURCE'] = 'DISTRIBUTOR_CREDIT'

In [86]:
# add columns to match d2 data
df_distributor_credits['CLAIMID'] = 'DC_' + df_distributor_credits['SALES_ORDER_ID'].astype(str)

df_distributor_credits['CLAIMPARTID'] = 0
df_distributor_credits['CLAIMPARTID'] = df_distributor_credits['CLAIMPARTID'].astype('int32')

df_distributor_credits['DATESUBMITTED'] = df_distributor_credits['INVOICE_DATE']

df_distributor_credits['PARTQTY'] = df_distributor_credits['INVOICE_QTY']

In [87]:
# rename columns from distributor credit table to match df_warranty_all
df_distributor_credits.rename(columns={"INVOICE_ID":"INVOICEID"},inplace=True)
df_distributor_credits.rename(columns={"SALES_ORDER_ID":"SO_ID"},inplace=True)
df_distributor_credits.rename(columns={"INVOICE_DATE":"INV_DATE"},inplace=True)
df_distributor_credits.rename(columns={"ITEM_STANDARD_COST_USD":"SO_LINE_ITEM_STD_COST_USD"},inplace=True)
df_distributor_credits.rename(columns={"INVOICE_QTY":"INV_LINE_INVOICED_QTY"},inplace=True)
df_distributor_credits.rename(columns={"INVOICED_AMOUNT_USD":"INV_LINE_AMT_USD"},inplace=True)

In [88]:
# split PRODUCT_KEY with hyphen and change to PARTNUMBER
#df_distributor_credits['PARTNUMBER'] = df_distributor_credits['PRODUCT_KEY'].str.split('-').str[1]


# Use a lambda function to apply the regex operation more explicitly
df_distributor_credits['PARTNUMBER'] = df_distributor_credits['PRODUCT_KEY'].apply(lambda x: re.sub(r'^[^-]*-', '', x))

# confirm split works
df_distributor_credits['PARTNUMBER'].value_counts(dropna=False)

PARTNUMBER
SP2670007X10       3599
SP0714T            2331
LACUS11100         2267
SW159412STL        1827
SP32950VSP         1777
                   ... 
SB1800FT18            1
SP3200UNKIT           1
TBH1100A0SV0000       1
ECX1037               1
HCXF2780003           1
Name: count, Length: 4407, dtype: int64

In [89]:
# add column for warranty cost
# note: std_cost is the cost for hayward to produce, invoiced amount is what is credited back
df_distributor_credits['DISTRIBUTOR_CREDIT_COST'] = df_distributor_credits['INV_LINE_AMT_USD']

In [90]:
# prepare to add column for country
df_distributor_credits['SHIP_TO_COUNTRY'].value_counts()

SHIP_TO_COUNTRY
United States of America    84131
Canada                      10098
France                       3483
Mexico                        610
Australia                     490
                            ...  
Brunei Darussalam               1
Pakistan                        1
Ireland                         1
Brazil                          1
USA-Latin America               1
Name: count, Length: 61, dtype: int64

In [91]:
country_region_dict = {
    'United States of America': 'USA',
    'Canada': 'CANADA',
    'France': 'EUROPE',
    'Mexico': 'LATIN AMERICA',
    'Australia': 'AUSTRALIA',
    'Spain': 'EUROPE',
    'Guadeloupe': 'LATIN AMERICA',
    'Italy': 'EUROPE',
    'Peru': 'LATIN AMERICA',
    'Belgium': 'EUROPE',
    'Dominican Republic': 'LATIN AMERICA',
    'Puerto Rico': 'USA', 
    'Costa Rica': 'LATIN AMERICA',
    'Portugal': 'EUROPE',
    'Trinidad and Tobago': 'LATIN AMERICA',
    'Netherlands': 'EUROPE',
    'Germany': 'EUROPE',
    'United Kingdom of Great Britain and Northern Ireland': 'EUROPE',
    'Martinique': 'LATIN AMERICA',
    'Saudi Arabia': 'MIDDLE EAST',
    'Ukraine': 'EUROPE',
    'Réunion': 'AFRICA',  
    'Chile': 'LATIN AMERICA',
    'Switzerland': 'EUROPE',
    'Philippines': 'ASIA',
    'Thailand': 'ASIA',
    'Colombia': 'LATIN AMERICA',
    'Panama': 'LATIN AMERICA',
    'Malaysia': 'ASIA',
    'Israel': 'MIDDLE EAST',
    'Ecuador': 'LATIN AMERICA',
    'Russian Federation': 'ASIA',
    'Romania': 'EUROPE',
    'Denmark': 'EUROPE',
    'Austria': 'EUROPE',
    'Honduras': 'LATIN AMERICA',
    'Turkey': 'EUROPE',
    'Sweden': 'EUROPE',
    'Egypt': 'MIDDLE EAST',
    'Singapore': 'ASIA',
    'Moldova, Republic of': 'EUROPE',
    'French Guiana': 'LATIN AMERICA',  
    'Algeria': 'AFRICA',
    'Cyprus': 'EUROPE',
    'Kuwait': 'MIDDLE EAST',
    'Japan': 'ASIA',
    'Argentina': 'LATIN AMERICA',
    'Croatia': 'EUROPE',
    'Indonesia': 'ASIA',
    'Poland': 'EUROPE',
    'Greece': 'EUROPE',
    'USA-Latin America': 'LATIN AMERICA',  # Special case, might need further clarification
    'Uzbekistan': 'ASIA',
    'Bolivia (Plurinational State of)': 'LATIN AMERICA',
    'Viet Nam': 'ASIA',
    'South Africa': 'AFRICA',
    'Pakistan': 'ASIA',
    'Ireland': 'EUROPE',
    'Brazil': 'LATIN AMERICA',
    'Bulgaria': 'EUROPE',
    'China': 'ASIA'  
}

In [92]:
# Function to determine country using state_dict
def determine_country_region(country_name):
    return country_region_dict.get(country_name, 'Other')

In [93]:
# add country column with function
df_distributor_credits['COUNTRY'] = df_distributor_credits['SHIP_TO_COUNTRY'].apply(determine_country_region)

In [94]:
df_distributor_credits.head()

,COMPANY_ID,INVOICEID,SO_ID,PRODUCT_KEY,CUSTOMER_SHIP_TO_KEY,INV_DATE,SO_LINE_ITEM_STD_COST_USD,SHIP_TO_CITY,SHIP_TO_STATE,SHIP_TO_COUNTRY,INV_LINE_INVOICED_QTY,INV_LINE_AMT_USD,WARRANTY_SOURCE,CLAIMID,CLAIMPARTID,DATESUBMITTED,PARTQTY,PARTNUMBER,DISTRIBUTOR_CREDIT_COST,COUNTRY
0,1,19202309,505528,1-S180T,1-175067-000105,2019-01-10,68.78,PLAINS,PENNSYLVANIA,United States of America,1,211.75,DISTRIBUTOR_CREDIT,DC_505528,0,2019-01-10,1,S180T,211.75,USA
1,1,19240427,760766,1-SP1580X15,1-175067-000007,2021-06-18,62.29,NASHVILLE,TENNESSEE,United States of America,1,203.06,DISTRIBUTOR_CREDIT,DC_760766,0,2021-06-18,1,SP1580X15,203.06,USA
2,1,19259448,868608,1-SP2670007X10,1-115600-000000,2022-09-15,203.19,SAGINAW,MICHIGAN,United States of America,1,478.33,DISTRIBUTOR_CREDIT,DC_868608,0,2022-09-15,1,SP2670007X10,478.33,USA
3,1,19240412,760785,1-SP0607U,1-179320-000000,2021-06-18,23.31,LOUISVILLE,KENTUCKY,United States of America,1,79.46,DISTRIBUTOR_CREDIT,DC_760785,0,2021-06-18,1,SP0607U,79.46,USA
4,1,19240411,760316,1-SP2670010X15,1-100009-000002,2021-06-18,125.22,PERRYSBURG,OHIO,United States of America,1,429.18,DISTRIBUTOR_CREDIT,DC_760316,0,2021-06-18,1,SP2670010X15,429.18,USA


# Company ID Table

In [95]:
df_company.head(0)

,COMPANY_KEY,COMPANY_ID,COMPANY_IS_ACTIVE_FLAG,COMPANY_ADDRESS_LINE_1,COMPANY_ADDRESS_LINE_2,COMPANY_LOCATION_NAME,COMPANY_DATE_FORMAT,COMPANY_SALES_CURRENCY_CD,COMPANY_DIVISION,COMPANY_COST_CURRENCY_CD,COMPANY_LAST_PRICE_CHANGE_DATE,COMPANY_CENTERED_NAME,COMPANY_NAME,COMPANY_PARENT_ID,COMPANY_PARENT_NAME,COMPANY_STATE_CD,COMPANY_DEFAULT_WAREHOUSE,COMPANY_ZIP,COMPANY_IS_IN_GDWH_FLAG,COMPANY_FISCAL_CALENDAR_MODEL,COMPANY_FISCAL_PERIOD_START_MONTH,COMPANY_GROUP_REGION,COMPANY_GROUP_LINE_OF_BUSINESS,COMPANY_REPORTED_NAME,COMPANY_MAIN_LOCATION_LONGITUDE_NUM,COMPANY_MAIN_LOCATION_LATITUDE_NUM


In [96]:
# remove unnecessary columns
columns_to_drop = ['COMPANY_KEY', 'COMPANY_ADDRESS_LINE_1', 'COMPANY_ADDRESS_LINE_2', 'COMPANY_LOCATION_NAME', 
                   'COMPANY_DATE_FORMAT', 'COMPANY_SALES_CURRENCY_CD', 'COMPANY_DIVISION', 'COMPANY_COST_CURRENCY_CD', 
                   'COMPANY_LAST_PRICE_CHANGE_DATE', 'COMPANY_NAME', 'COMPANY_CENTERED_NAME', 'COMPANY_PARENT_ID', 
                   'COMPANY_PARENT_NAME', 'COMPANY_DEFAULT_WAREHOUSE', 'COMPANY_ZIP', 'COMPANY_IS_IN_GDWH_FLAG', 
                   'COMPANY_FISCAL_CALENDAR_MODEL', 'COMPANY_FISCAL_PERIOD_START_MONTH', 'COMPANY_GROUP_REGION', 
                   'COMPANY_GROUP_LINE_OF_BUSINESS', 'COMPANY_MAIN_LOCATION_LONGITUDE_NUM', 
                   'COMPANY_MAIN_LOCATION_LATITUDE_NUM']

df_company.drop(columns=columns_to_drop, inplace=True)

In [97]:
# only active companies
df_company_active = df_company[df_company['COMPANY_IS_ACTIVE_FLAG'] == 'YES']
df_company_active

,COMPANY_ID,COMPANY_IS_ACTIVE_FLAG,COMPANY_STATE_CD,COMPANY_REPORTED_NAME
1,30,YES,TX,"30 - Core Industrial Products, TX"
2,31,YES,IN,"31 - Fwdc- Harrington Industrial, IN"
3,32,YES,MI,"32 - Tro Sales--Industrial Products, MI"
4,50,YES,LA,"50 - Hayward Ind.-New Orleans Whse, LA"
5,51,YES,TX,"51 - Hayward Ind.-Houston Whse, TX"
20,33,YES,None,"33 - Hayward Industrial Southeast,"
24,17,YES,CN,"17 - Hayward Industrial, Canada, CN"
25,19,YES,FO,"19 - Melbourne, Australia"
26,777,YES,NJ,"777 - Hayward Master Files, NJ"
27,21,YES,FO,"21 - Kripsol, Spain"


In [98]:
# assign country to companies with 'FO' code (stands for foreign) 
df_company_active.loc[df_company_active['COMPANY_ID'] == 33, 'COMPANY_STATE_CD'] = 'USA'
df_company_active.loc[df_company_active['COMPANY_ID'] == 19, 'COMPANY_STATE_CD'] = 'AUS'
df_company_active.loc[df_company_active['COMPANY_ID'] == 21, 'COMPANY_STATE_CD'] = 'ESP'
df_company_active.loc[df_company_active['COMPANY_ID'] == 22, 'COMPANY_STATE_CD'] = 'ESP'
df_company_active.loc[df_company_active['COMPANY_ID'] == 18, 'COMPANY_STATE_CD'] = 'CHN'
df_company_active.loc[df_company_active['COMPANY_ID'] == 16, 'COMPANY_STATE_CD'] = 'FRA'
df_company_active.loc[df_company_active['COMPANY_ID'] == 4, 'COMPANY_STATE_CD'] = 'CHN'
df_company_active.loc[df_company_active['COMPANY_ID'] == 26, 'COMPANY_STATE_CD'] = 'UNKNOWN'

In [99]:
# create a dictionary to be used for creating a country column in the data set
state_dict = {
    'AL': 'USA', 'AK': 'USA', 'AZ': 'USA', 'AR': 'USA', 'CA': 'USA',
    'CO': 'USA', 'CT': 'USA', 'DE': 'USA', 'FL': 'USA', 'GA': 'USA',
    'HI': 'USA', 'ID': 'USA', 'IL': 'USA', 'IN': 'USA', 'IA': 'USA',
    'KS': 'USA', 'KY': 'USA', 'LA': 'USA', 'ME': 'USA', 'MD': 'USA',
    'MA': 'USA', 'MI': 'USA', 'MN': 'USA', 'MS': 'USA', 'MO': 'USA',
    'MT': 'USA', 'NE': 'USA', 'NV': 'USA', 'NH': 'USA', 'NJ': 'USA',
    'NM': 'USA', 'NY': 'USA', 'NC': 'USA', 'ND': 'USA', 'OH': 'USA',
    'OK': 'USA', 'OR': 'USA', 'PA': 'USA', 'RI': 'USA', 'SC': 'USA',
    'SD': 'USA', 'TN': 'USA', 'TX': 'USA', 'UT': 'USA', 'VT': 'USA',
    'VA': 'USA', 'WA': 'USA', 'WV': 'USA', 'WI': 'USA', 'WY': 'USA', 'PR': 'USA',
    'CN': 'CANADA', 'USA': 'USA', 'AUS': 'AUSTRALIA', 'ESP': 'EUROPE', 'CHN': 'ASIA', 'FRA': 'EUROPE', 
    'ON': 'CANADA', 'QC': 'CANADA', 'BC': 'CANADA', 'MB': 'CANADA', 'NS': 'CANADA', 'AB': 'CANADA', 'CN': 'CANADA',
}

In [100]:
# Function to determine country using state_dict
def determine_country(state_code):
    return state_dict.get(state_code, 'UNKNOWN')

In [101]:
# add country column with function
df_company_active['COUNTRY'] = df_company_active['COMPANY_STATE_CD'].apply(determine_country)

C:\Users\mherndon\AppData\Local\Temp\ipykernel_13300\721937120.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_company_active['COUNTRY'] = df_company_active['COMPANY_STATE_CD'].apply(determine_country)


# Product

In [102]:
df_product.tail()

,ITEM_KEY,COMPANY_ID,ITEM_ID,CORE_ITEM_ID,B_AND_B_NAME,ITEM_WEB_FLAG,ITEM_DESC,ITEM_ABC_CD,ITEM_BUYER_PLANNER_CD,ITEM_COMMODITY_CD,ITEM_COMMODITY_DESC,ITEM_LEAD_TIME_DAYS,ITEM_MAKE_BUY_CD,ITEM_MAKE_BUY_DESC,GLOBAL_ITEM_MAKE_BUY_DESC,ITEM_PREVIOUS_CURRENT_COST,ITEM_STANDARD_COST,ITEM_STANDARD_COST_USD,ITEM_LOT_SIZE,ITEM_PRODUCT_FAMILY_GROUP,ITEM_CLASS_CODE,ITEM_CLASS_DESC,ITEM_SAFETY_STOCK_QTY,ITEM_STATUS_CODE,ITEM_STATUS_DESC,ITEM_UNIT_OF_MEASURE_CD,ITEM_UNIT_OF_MEASURE_DESC,ITEM_SOURCE_COMPANY_ID,ITEM_MANUFACTURING_COMPANY_ID,ITEM_IS_PRIVATE_LABEL,ITEM_FAMILY_GROUPING_CD,ITEM_FAMILY_MARKET_DESC,ITEM_FAMILY_APPLICATION_DESC,ITEM_FAMILY_TYPE_DESC,ITEM_FAMILY_CATEGORY_DESC,ITEM_FAMILY_ATTRIBUTE_DESC,ITEM_CIA_CD,ITEM_CIA_DESC,ITEM_LIST_PRICE_EFFECTIVE_DATE,ITEM_FORECAST_FAMILY_CD,ITEM_FORECAST_FAMILY_DESC,ITEM_TYPE_CD,ITEM_TYPE_DESC,ITEM_FAMILY_GROUPING_6_CD,ITEM_SHORT_DESC,ITEM_MATERIAL_STANDARD_COST_AMT,INTERCO_MARKUP_DECREASE_RATE,SOURCE_SYSTEM_FLAG
337362,8030405813904568161,1,096206B-12,096206B-12,US Pools,NO,WORK INSTR SW-5PK,None,GL4,NCR,None,0.00,IR,BUY,BUY,"20,240,728.00",0.00,0.00,1.00,None,875,GOLDLINE COMPONENTS,0.00,C,TO BE OBSOLETED,EA,EACH,9.00,9.00,None,None,None,None,None,None,None,None,None,None,5Q,GOLDLINE COMPONENTS,C,COMPONENT,None,WORK INSTR SW-5PK,0.00,NaN,USA
337363,7029421455263347822,8,SB1150STE132,SB1150STE132,Flow Control,NO,"1 1/2"" SIMPLEX PVC EPDM ST 1/32 PERF",None,STR,FGP,None,0.00,IR,BUY,MAKE,NaN,132.63,132.63,1.00,002,522,PVC BASKET STRAINERS & BASKETS,0.00,A,ACTIVE,EA,EACH,3.00,3.00,None,IF09F06197,INDUSTRIAL,FLOW CONTROL,FILTRATION SYSTEMS,STRAINERS,SB SERIES,None,None,2024-05-01,20,STRAINER & FILTRATION,F,FINISHED GOOD,None,"1 1/2"" SIMPLEX PVC EPDM ST 1/3",0.00,NaN,USA
337364,2728814039845790145,22,RC175,RC175,Europe,NO,CELL 175GR,0,OTROS,None,None,NaN,IF,MAKE,MAKE,"20,240,114.00",896.32,973.08,NaN,None,None,None,NaN,A,ACTIVE,EA,EACH,NaN,NaN,None,CI05S03194,COMMERCIAL,IN GROUND,CHEM AUTOMATION & SANITIZATION,SALT,SALT CELL,C,Comercial,None,None,None,F,FINISHED GOOD,TB550V,None,NaN,NaN,EUR
337365,7594009363390612652,4,PHN19-725-102,PHN19-725-102,OTHER,NO,PHIN INLINE SHELL OUTER BACK,None,SP3,MP,MOLDED PARTS,75.00,IF,MAKE,MAKE,NaN,4.40,0.61,"2,500.00",None,190,S/P PARTS,0.00,A,ACTIVE,EA,EACH,4.00,4.00,None,None,None,None,None,None,None,None,None,None,51,CONNECTED YARD PHIN,C,COMPONENT,None,PHIN INLINE SHELL OUTER BACK,0.00,NaN,USA
337366,5391023407881175183,22,RIN4110,RIN4110,Europe,NO,None,0,OTROS,None,None,NaN,IR,BUY,BUY,"20,230,110.00",0.00,0.00,NaN,None,None,None,NaN,B,INACTIVE,EA,EACH,NaN,NaN,None,RI05A01040,RESIDENTIAL,IN GROUND,CHEM AUTOMATION & SANITIZATION,REPLACEMENT PARTS,CHEM AUTOMATION ACCESSORIES,I,Residencial - In Ground,None,None,None,S,SPARE PART,TA000V,None,NaN,NaN,EUR


In [103]:
# Convert COMPANY_ID and ITEM_ID to string
df_product['COMPANY_ID'] = df_product['COMPANY_ID'].astype(str)
df_product['ITEM_ID'] = df_product['ITEM_ID'].astype(str)

In [104]:
# create product key
df_product['PRODUCT_KEY'] = ( df_product['COMPANY_ID'].str.strip().str.upper() + '-' + 
                             df_product['ITEM_ID'].str.strip().str.upper() )

# make sure this is formatted exactly to ensure merging works as intended
df_product['PRODUCT_KEY'] = df_product['PRODUCT_KEY'].str.strip().str.upper().astype(str)

In [105]:
# remove unnecessary columns
columns_to_drop = ['COMPANY_ID', 'ITEM_ID', 'ITEM_KEY', 'CORE_ITEM_ID', 'ITEM_WEB_FLAG', 'ITEM_ABC_CD', 
                   'ITEM_BUYER_PLANNER_CD', 'ITEM_COMMODITY_CD', 'ITEM_COMMODITY_DESC', 'ITEM_LEAD_TIME_DAYS', 
                   'ITEM_MAKE_BUY_CD', 'ITEM_PREVIOUS_CURRENT_COST', 'ITEM_STANDARD_COST', 
                   'ITEM_LOT_SIZE', 'ITEM_PRODUCT_FAMILY_GROUP', 'ITEM_CLASS_CODE', 'ITEM_CLASS_DESC', 
                   'ITEM_SAFETY_STOCK_QTY', 'ITEM_STATUS_CODE', 'ITEM_STATUS_DESC', 
                   'ITEM_UNIT_OF_MEASURE_CD', 'ITEM_UNIT_OF_MEASURE_DESC', 'ITEM_SOURCE_COMPANY_ID',
                   'ITEM_IS_PRIVATE_LABEL', 'ITEM_FAMILY_GROUPING_CD', 
                   'ITEM_FAMILY_MARKET_DESC', 'ITEM_FAMILY_APPLICATION_DESC', 'ITEM_FAMILY_ATTRIBUTE_DESC', 'ITEM_CIA_CD', 
                   'ITEM_CIA_DESC', 'ITEM_LIST_PRICE_EFFECTIVE_DATE', 'ITEM_FORECAST_FAMILY_CD', 'ITEM_TYPE_CD', 
                   'ITEM_TYPE_DESC', 'ITEM_FAMILY_GROUPING_6_CD', 'ITEM_SHORT_DESC', 'ITEM_MATERIAL_STANDARD_COST_AMT', 
                   'INTERCO_MARKUP_DECREASE_RATE', 'SOURCE_SYSTEM_FLAG']

df_product.drop(columns=columns_to_drop, inplace=True)

In [106]:
df_product.tail()

,B_AND_B_NAME,ITEM_DESC,ITEM_MAKE_BUY_DESC,GLOBAL_ITEM_MAKE_BUY_DESC,ITEM_STANDARD_COST_USD,ITEM_MANUFACTURING_COMPANY_ID,ITEM_FAMILY_TYPE_DESC,ITEM_FAMILY_CATEGORY_DESC,ITEM_FORECAST_FAMILY_DESC,PRODUCT_KEY
337362,US Pools,WORK INSTR SW-5PK,BUY,BUY,0.00,9.00,None,None,GOLDLINE COMPONENTS,1-096206B-12
337363,Flow Control,"1 1/2"" SIMPLEX PVC EPDM ST 1/32 PERF",BUY,MAKE,132.63,3.00,FILTRATION SYSTEMS,STRAINERS,STRAINER & FILTRATION,8-SB1150STE132
337364,Europe,CELL 175GR,MAKE,MAKE,973.08,NaN,CHEM AUTOMATION & SANITIZATION,SALT,None,22-RC175
337365,OTHER,PHIN INLINE SHELL OUTER BACK,MAKE,MAKE,0.61,4.00,None,None,CONNECTED YARD PHIN,4-PHN19-725-102
337366,Europe,None,BUY,BUY,0.00,NaN,CHEM AUTOMATION & SANITIZATION,REPLACEMENT PARTS,None,22-RIN4110


In [107]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337367 entries, 0 to 337366
Data columns (total 10 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   B_AND_B_NAME                   337367 non-null  object 
 1   ITEM_DESC                      302866 non-null  object 
 2   ITEM_MAKE_BUY_DESC             332345 non-null  object 
 3   GLOBAL_ITEM_MAKE_BUY_DESC      334807 non-null  object 
 4   ITEM_STANDARD_COST_USD         337059 non-null  float64
 5   ITEM_MANUFACTURING_COMPANY_ID  284617 non-null  float64
 6   ITEM_FAMILY_TYPE_DESC          98817 non-null   object 
 7   ITEM_FAMILY_CATEGORY_DESC      98797 non-null   object 
 8   ITEM_FORECAST_FAMILY_DESC      282450 non-null  object 
 9   PRODUCT_KEY                    337367 non-null  object 
dtypes: float64(2), object(8)
memory usage: 25.7+ MB


In [108]:
# is PRODUCT_KEY unique?
print('Total Unique Records:', df_product['PRODUCT_KEY'].nunique())
print('Total Length of Records:', len(df_product['PRODUCT_KEY']))
print('Is field unique:', df_product['PRODUCT_KEY'].is_unique)

Total Unique Records: 337367
Total Length of Records: 337367
Is field unique: True


# Sales Orders

In [109]:
columns_to_drop = ['PARENT_COMPANY_ID', 'BILL_CUST_NBR', 'BILL_CUST_NBR_SEQ', 'SHIP_CUST_NBR', 'SHIP_CUST_NBR_SEQ', 
               'SO_TAKEN_CALENDAR_KEY', 'ITEM_ID', 'SO_HDR_DATE', 'SO_NEXT_SHIPMENT_SEQ', 
               'SO_SPLIT_PARENT_COMPANY_ID', 'SO_SIBLING_COMPANY_ID', 'SO_CURRENCY_CD', 'SO_LAST_INVOICE_DATE', 
               'SO_IS_DROP_SHIPMENT', 'SO_IS_SPLIT', 'SO_IS_INTERCOMPANY', 'SO_FOB_CD', 
               'SO_FOB_DESC', 'SO_REASON_CD', 'SO_REASON_DESC', 'SO_IS_INTERNATIONAL', 'SO_CUST_ACCT_BILL_TO_ID', 
               'SO_CUST_ACCT_SHIP_TO_ID', 'SO_PARENT_OF_CUST_ACCT_BILL_TO_ID', 'SO_PARENT_OF_CUST_ACCT_SHIP_TO_ID', 
               'SO_PARENT_SO_ID', 'SO_SIBLING_SO_ID', 'SO_CR_FAILURE_REASON_CD', 
               'SO_CR_FAILURE_REASON_DESC', 'SO_PARENT_COMPANY_ID', 'SO_SHIP_TO_ZIP', 'SO_SHIP_TO_CITY', 
               'SO_SHIP_TO_PROVINCE_CD', 'SO_SHIP_TO_PROVINCE_DESC', 'SO_SHIP_TO_STATE_DESC', 'SO_SHIP_TO_COUNTRY_DESC',
               'SO_SHIP_TO_REGION_CD', 'SO_SHIP_TO_REGION_DESC', 'SO_SHIP_TO_TERRITORY_CD', 'SO_SHIP_TO_TERRITORY_DESC',
               'SO_SALES_PROGRAM_DESC', 'SO_SALES_PROGRAM_CD', 'SO_SALES_PROGRAM_YEAR', 'SO_STATUS_CD',
               'SO_STATUS_DESC', 'SO_SHIP_FROM_WAREHOUSE_CD', 'SO_TOTAL_ORDERED_ADJUSTMENT_AMT', 'SO_TOTAL_ORDERED_NET_AMT',
               'SO_TOTAL_ORDERED_ITEMS_AMT', 'SO_NUM_LINES_SHIPPED_QTY', 'SO_SHIPPED_TO_DATE_ITEMS_QTY', 
               'SO_SHIPPED_TO_DATE_ITEMS_AMT', 'SO_TOTAL_TAX_AMT', 'SO_TOTAL_TAXABLE_AMT', 'SO_WAITING_SHIPMENT_QTY', 
               'SO_SHIPMENT_CONDITION_CD', 'SO_SHIPMENT_CONDITION_DESC', 'SO_TERMS_CD', 'SO_TERMS_DESC', 'SO_IS_QTY_COUNTED', 
               'SO_IS_AMT_COUNTED', 'SO_LINE_CUST_REQUEST_DATE', 'SO_LINE_EXTENDED_AFTER_DISCOUNT_PRICE', 
               'SO_LINE_LAST_SHIPMENT_QTY', 'SO_LINE_INVENTORY_ISSUED_QTY', 'SO_LINE_CREDIT_REASON_CD', 
               'SO_LINE_SHIPPED_TO_DATE_QTY', 'SO_LINE_STATUS_CD', 'SO_LINE_ITEM_IS_TAXABLE', 
               'SO_LINE_DISCOUNTED_UNIT_PRICE', 'SO_LINE_DISCOUNTED_UNIT_PRICE_USD', 
               'SO_LINE_ORIGINAL_PROMISE_DATE', 'SO_LINE_CURRENT_PROMISE_DATE', 'SOURCE_SYSTEM_FLAG', 
               'SO_TYPE_CLASSIFICATION_CD', 'SHIP_TO_CUSTOMER_PARENT_KEY', 'BILL_TO_CUSTOMER_PARENT_KEY', 
               'CUSTOMER_PARENT_KEY', 'SO_LINE_ITEM_LIST_UNIT_PRICE']

df_sales_order.drop(columns=columns_to_drop, inplace=True)

In [110]:
df_sales_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4903958 entries, 0 to 4903957
Data columns (total 20 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   COMPANY_ID                  int8   
 1   SO_TAKEN_DATE               int32  
 2   SO_ID                       object 
 3   SO_LINE_NUM                 float64
 4   SO_LINE_ITEM_ID             object 
 5   SO_DATE                     object 
 6   SO_PURCHASE_ORDER_ID        object 
 7   SO_LAST_SHIPMENT_DATE       object 
 8   SO_TYPE_CD                  object 
 9   SO_TYPE_DESC                object 
 10  SO_SHIP_TO_STATE_CD         object 
 11  SO_SHIP_TO_COUNTRY_CD       object 
 12  SO_LINE_REVENUE_USD         float64
 13  SO_TOTAL_ORDERED_ITEMS_QTY  float64
 14  SO_SHIP_TO_ACCT_NAME        object 
 15  SO_LINE_ITEM_STD_COST       float64
 16  SO_LINE_ITEM_STD_COST_USD   float64
 17  SO_LINE_TOTAL_INVOICED_QTY  float64
 18  SO_LINE_ORDERED_QTY         int32  
 19  SO_LINE_ITEM_LIST_PRI

In [111]:
# sales order table
df_sales_order.tail()

,COMPANY_ID,SO_TAKEN_DATE,SO_ID,SO_LINE_NUM,SO_LINE_ITEM_ID,SO_DATE,SO_PURCHASE_ORDER_ID,SO_LAST_SHIPMENT_DATE,SO_TYPE_CD,SO_TYPE_DESC,SO_SHIP_TO_STATE_CD,SO_SHIP_TO_COUNTRY_CD,SO_LINE_REVENUE_USD,SO_TOTAL_ORDERED_ITEMS_QTY,SO_SHIP_TO_ACCT_NAME,SO_LINE_ITEM_STD_COST,SO_LINE_ITEM_STD_COST_USD,SO_LINE_TOTAL_INVOICED_QTY,SO_LINE_ORDERED_QTY,SO_LINE_ITEM_LIST_PRICE
4903953,16,20190528,86570,4.00,CLX220J,2019-05-28,CDE WEB27.05.19-11218,None,C,Regular Sale,FO,FRA,3.58,25.00,CAMPOY SARL,0.94,1.05,1.00,1,8.00
4903954,6,20190506,956390,9.00,SP3215X20,2019-05-06,BG270714,None,C,Regular Sale,TX,USA,500.12,112.00,"SCP DISTRIBUTORS, LLC",159.36,159.36,1.00,1,"1,231.44"
4903955,9,20230926,598535,9.00,SP0583L30,2023-09-26,TN054627,None,E,Early Buy,NJ,USA,"3,538.72",77.00,"JETLINE-SCP DISTRIBUTORS, LLC",85.15,85.15,17.00,17,512.56
4903956,1,20210415,739805,1.00,SPX1500P,2021-04-15,AQT823422,None,C,Regular Sale,SC,USA,25.32,21.00,"PARADISE POOLS & SPAS, INC.",0.60,0.60,6.00,6,8.89
4903957,1,20230306,900609,118.00,SPX1082KDGR,2023-03-06,05IC4717,None,C,Regular Sale,CT,USA,558.00,"9,778.00",AQUATIC PARTS COMPANY,2.04,2.04,50.00,50,27.48


In [112]:
# rename columns so as not to conflict with columnx of same name in df_warranty_claims_parts
df_sales_order.rename(columns={"COMPANY_ID":"COMPANY_ID_FOR_SO"},inplace=True)
df_sales_order.rename(columns={"SO_ID":"SO_ID_FOR_SO"},inplace=True)
df_sales_order.rename(columns={"SO_LINE_NUM":"SO_LINE_NUM_FOR_SO"},inplace=True)

In [113]:
# format
df_sales_order['SO_LINE_ITEM_STD_COST_USD']  = df_sales_order['SO_LINE_ITEM_STD_COST_USD'].astype('float64')

In [114]:
# create a calc for the total cost of the sales order line
df_sales_order['SO_LINE_ITEM_TOTAL_COST_CALC'] = ( 
                                                df_sales_order['SO_LINE_TOTAL_INVOICED_QTY'] * 
                                                df_sales_order['SO_LINE_ITEM_STD_COST_USD'] 
                                                 )

In [115]:
# only need warranty orders
df_sales_warranty = df_sales_order[df_sales_order['SO_TYPE_CD'] == 'D'].copy()

# Table of Sales Order Types provided by Tom Emanuele 12/8/23
#    1     &    RESERVED            
#    1     #    REPLIC. TO BILL     
#    1     @    RESERVED            
#    1     A    NO CHARGE           
#    1     B    CONSIGNMENT         
#    1     C    NORMAL NEW SALE     
#    1     D    WARRANTY            
#    1     E    EARLY BUY           
#    1     F    BILLING ONLY        
#    1     G    INTER-PLANT         
#    1     H    DISTRIBUTION        
#    1     I    C.I.D.              
#    1     K    LENTZ SHIPMENT      
#    1     L    BILL BACK           
#    1     M    MANUAL WARRANTY

In [116]:
# SO_ID_FOR_SO is not a unique identifier but we can aggregate to make a unique key
df_sales_warranty_aggregate = df_sales_warranty.groupby(['SO_ID_FOR_SO', 'COMPANY_ID_FOR_SO', 'SO_LINE_ITEM_ID', 'SO_DATE', 
                                             'SO_TYPE_CD', 'SO_TYPE_DESC', 'SO_SHIP_TO_STATE_CD',
                                             'SO_SHIP_TO_COUNTRY_CD', 'SO_SHIP_TO_ACCT_NAME']).agg({
                'SO_LINE_TOTAL_INVOICED_QTY': 'sum',
                'SO_LINE_ITEM_STD_COST_USD': 'mean',
                'SO_LINE_ITEM_TOTAL_COST_CALC': 'sum'
}).reset_index()

In [117]:
# find a duplicate to see 'SO_LINE_ITEM_STD_COST_USD' mean calc and compare to df_sales_aggregate
df_sales_warranty_duplicates = df_sales_warranty[df_sales_warranty.duplicated(
                        subset=[
                        'SO_ID_FOR_SO', 
                        'SO_LINE_ITEM_ID',
                        'COMPANY_ID_FOR_SO'
                         ], keep=False)].sort_values('SO_ID_FOR_SO', ascending=True)

# order 647644 was identified as a duplicate previously
print(df_sales_warranty_duplicates[df_sales_warranty_duplicates['SO_ID_FOR_SO'] =='647644']['SO_LINE_TOTAL_INVOICED_QTY'].sum())
df_sales_warranty_duplicates[df_sales_warranty_duplicates['SO_ID_FOR_SO'] == '647644']

1584.0


,COMPANY_ID_FOR_SO,SO_TAKEN_DATE,SO_ID_FOR_SO,SO_LINE_NUM_FOR_SO,SO_LINE_ITEM_ID,SO_DATE,SO_PURCHASE_ORDER_ID,SO_LAST_SHIPMENT_DATE,SO_TYPE_CD,SO_TYPE_DESC,SO_SHIP_TO_STATE_CD,SO_SHIP_TO_COUNTRY_CD,SO_LINE_REVENUE_USD,SO_TOTAL_ORDERED_ITEMS_QTY,SO_SHIP_TO_ACCT_NAME,SO_LINE_ITEM_STD_COST,SO_LINE_ITEM_STD_COST_USD,SO_LINE_TOTAL_INVOICED_QTY,SO_LINE_ORDERED_QTY,SO_LINE_ITEM_LIST_PRICE,SO_LINE_ITEM_TOTAL_COST_CALC
797618,1,20200608,647644,35.00,ECX2712B1,2020-06-08,Springs Pools & Spas/AL,None,D,WARRANTY,WI,USA,0.00,"1,584.00","Spring's Pools & Spas, LLC",2.53,2.53,12.00,12,23.61,30.34
4379707,1,20200608,647644,132.00,ECX2712B1,2020-06-08,Springs Pools & Spas/AL,None,D,WARRANTY,WI,USA,0.00,"1,584.00","Spring's Pools & Spas, LLC",2.53,2.53,12.00,12,23.61,30.34
1864327,1,20200608,647644,66.00,ECX2712B1,2020-06-08,Springs Pools & Spas/AL,None,D,WARRANTY,WI,USA,0.00,"1,584.00","Spring's Pools & Spas, LLC",2.53,2.53,12.00,12,23.61,30.34
2364209,1,20200608,647644,100.00,ECX2712B1,2020-06-08,Springs Pools & Spas/AL,None,D,WARRANTY,WI,USA,0.00,"1,584.00","Spring's Pools & Spas, LLC",2.53,2.53,12.00,12,23.61,30.34
912180,1,20200608,647644,44.00,ECX2712B1,2020-06-08,Springs Pools & Spas/AL,None,D,WARRANTY,WI,USA,0.00,"1,584.00","Spring's Pools & Spas, LLC",2.53,2.53,12.00,12,23.61,30.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3894956,1,20200608,647644,65.00,ECX2712B1,2020-06-08,Springs Pools & Spas/AL,None,D,WARRANTY,WI,USA,0.00,"1,584.00","Spring's Pools & Spas, LLC",2.53,2.53,12.00,12,23.61,30.34
3909011,1,20200608,647644,128.00,ECX2712B1,2020-06-08,Springs Pools & Spas/AL,None,D,WARRANTY,WI,USA,0.00,"1,584.00","Spring's Pools & Spas, LLC",2.53,2.53,12.00,12,23.61,30.34
3920649,1,20200608,647644,122.00,ECX2712B1,2020-06-08,Springs Pools & Spas/AL,None,D,WARRANTY,WI,USA,0.00,"1,584.00","Spring's Pools & Spas, LLC",2.53,2.53,12.00,12,23.61,30.34
2247247,1,20200608,647644,116.00,ECX2712B1,2020-06-08,Springs Pools & Spas/AL,None,D,WARRANTY,WI,USA,0.00,"1,584.00","Spring's Pools & Spas, LLC",2.53,2.53,12.00,12,23.61,30.34


In [118]:
# compare 'SO_LINE_ITEM_STD_COST_USD' mean calc in df_sales_aggregate
df_sales_warranty_aggregate[df_sales_warranty_aggregate['SO_ID_FOR_SO'] == '647644']

,SO_ID_FOR_SO,COMPANY_ID_FOR_SO,SO_LINE_ITEM_ID,SO_DATE,SO_TYPE_CD,SO_TYPE_DESC,SO_SHIP_TO_STATE_CD,SO_SHIP_TO_COUNTRY_CD,SO_SHIP_TO_ACCT_NAME,SO_LINE_TOTAL_INVOICED_QTY,SO_LINE_ITEM_STD_COST_USD,SO_LINE_ITEM_TOTAL_COST_CALC
355851,647644,1,ECX2712B1,2020-06-08,D,WARRANTY,WI,USA,"Spring's Pools & Spas, LLC","1,584.00",2.53,"4,005.14"


In [119]:
# save memory
del df_sales_warranty_duplicates

In [120]:
# create a key for joining later on
df_sales_warranty_aggregate['SO_COMPANY_ITEM'] = (df_sales_warranty_aggregate['SO_ID_FOR_SO'].astype(str)
                                + '_' 
                                + df_sales_warranty_aggregate['COMPANY_ID_FOR_SO'].fillna(0).astype(float).round(1).astype(str)
                               + '_' 
                                + df_sales_warranty_aggregate['SO_LINE_ITEM_ID'].astype(str) )

In [121]:
# is SO_COMPANY_ITEM unique?
print('Total Unique Records:', df_sales_warranty_aggregate['SO_COMPANY_ITEM'].nunique())
print('Total Length of Records:', len(df_sales_warranty_aggregate['SO_COMPANY_ITEM']))
print('Is field unique:', df_sales_warranty_aggregate['SO_COMPANY_ITEM'].is_unique)

Total Unique Records: 537204
Total Length of Records: 537204
Is field unique: True


In [122]:
# join test
# df_merge_test_5 = pd.merge(df_warranty_claims_parts_all, df_sales_warranty_aggregate, 
#                            on='SO_COMPANY_ITEM', how='left')
# df_merge_test_5.head()

In [123]:
# join successful, save memory
# del df_merge_test_5

# No Charge Orders

No charge orders are sales orders marked 'A' type which classifies it as a no charge order.  These orders are treated separately from the D2 claims.

In [124]:
# create new dataframe
df_nocharge_prelim = df_sales_order[df_sales_order['SO_TYPE_CD'] == 'D'].copy()
df_nocharge_prelim.reset_index(drop=True, inplace=True)

# Table of Sales Order Types provided by Tom Emanuele 12/8/23
#    1     &    RESERVED            
#    1     #    REPLIC. TO BILL     
#    1     @    RESERVED            
#    1     A    NO CHARGE           
#    1     B    CONSIGNMENT         
#    1     C    NORMAL NEW SALE     
#    1     D    WARRANTY            
#    1     E    EARLY BUY           
#    1     F    BILLING ONLY        
#    1     G    INTER-PLANT         
#    1     H    DISTRIBUTION        
#    1     I    C.I.D.              
#    1     K    LENTZ SHIPMENT      
#    1     L    BILL BACK           
#    1     M    MANUAL WARRANTY

In [125]:
# remove unnecessary columns
columns_to_drop = ['SO_PURCHASE_ORDER_ID', 'SO_SHIP_TO_COUNTRY_CD']

df_nocharge_prelim.drop(columns=columns_to_drop, inplace=True)

In [126]:
# rename columns to match up with data it will be appended to
df_nocharge_prelim.rename(columns={"COMPANY_ID_FOR_SO":"COMPANY_ID"},inplace=True)
df_nocharge_prelim.rename(columns={"SO_TAKEN_DATE":"DATESUBMITTED"},inplace=True)
df_nocharge_prelim.rename(columns={"SO_ID_FOR_SO":"SO_ID"},inplace=True)
df_nocharge_prelim.rename(columns={"SO_LINE_NUM_FOR_SO":"POSITIONID"},inplace=True)
df_nocharge_prelim.rename(columns={"SO_LINE_ITEM_ID":"PARTNUMBER"},inplace=True)
df_nocharge_prelim.rename(columns={"SO_LAST_SHIPMENT_DATE":"SHIPMENT_DATE"},inplace=True)
df_nocharge_prelim.rename(columns={"SO_DATE":"ORDERDATE"},inplace=True)
df_nocharge_prelim.rename(columns={"SO_SHIP_TO_ACCT_NAME":"SERVICINGCOMPANY"},inplace=True)

In [127]:
df_nocharge_prelim.tail()

,COMPANY_ID,DATESUBMITTED,SO_ID,POSITIONID,PARTNUMBER,ORDERDATE,SHIPMENT_DATE,SO_TYPE_CD,SO_TYPE_DESC,SO_SHIP_TO_STATE_CD,SO_LINE_REVENUE_USD,SO_TOTAL_ORDERED_ITEMS_QTY,SERVICINGCOMPANY,SO_LINE_ITEM_STD_COST,SO_LINE_ITEM_STD_COST_USD,SO_LINE_TOTAL_INVOICED_QTY,SO_LINE_ORDERED_QTY,SO_LINE_ITEM_LIST_PRICE,SO_LINE_ITEM_TOTAL_COST_CALC
554437,9,20230605,582261,1.00,GLX-TROL-TIMER,2023-06-05,None,D,WARRANTY,NJ,0.00,1.00,Jasmine Medrano,24.66,24.66,1.00,1,142.53,24.66
554438,1,20201026,697364,1.00,SPX3206Z1VSPE,2020-10-26,None,D,WARRANTY,FL,0.00,1.00,POOL BOY INC.,446.29,446.29,1.00,1,"2,081.67",446.29
554439,6,20191018,976672,2.00,GLX-CELL-5-W,2019-10-18,None,D,WARRANTY,AL,0.00,2.00,BLUE HAVEN POOLS & SPAS- MOBIL,81.83,81.83,1.00,1,756.52,81.83
554440,60,20230629,345574,1.00,SPX3206Z1VSPE,2023-06-29,None,D,WARRANTY,CA,0.00,1.00,CESAR'S POOL REPAIR,509.86,509.86,1.00,1,"2,865.08",509.86
554441,1,20230619,933518,1.00,HPX11023693,2023-06-19,None,D,WARRANTY,FL,0.00,11.00,Bob Corsetti,9.12,9.12,1.00,1,71.03,9.12


In [128]:
# need to aggregate no charge orders since there are multiple combinations of SO_ID and PARTNUMBER due to POSITIONID
df_nocharge = df_nocharge_prelim.groupby(['SO_ID', 'PARTNUMBER', 'COMPANY_ID', 'SERVICINGCOMPANY']).agg({
                                               'DATESUBMITTED': 'first',
                                               'ORDERDATE': 'first',
                                               'SHIPMENT_DATE': 'first', 
                                               'SO_LINE_TOTAL_INVOICED_QTY': 'sum',
                                               'SO_LINE_ITEM_STD_COST_USD': 'mean',
                                                'SO_LINE_ITEM_TOTAL_COST_CALC': 'sum'
                                                }).reset_index()

In [129]:
# confirm there are not multiple sales orders / part number / company id combinations
df_nocharge_duplicates = df_nocharge[df_nocharge.duplicated(
                        subset=['SO_ID', 'PARTNUMBER', 'COMPANY_ID'], keep=False)].sort_values('SO_ID', ascending=True)
print(df_nocharge_duplicates.head())
del df_nocharge_duplicates

Empty DataFrame
Columns: [SO_ID, PARTNUMBER, COMPANY_ID, SERVICINGCOMPANY, DATESUBMITTED, ORDERDATE, SHIPMENT_DATE, SO_LINE_TOTAL_INVOICED_QTY, SO_LINE_ITEM_STD_COST_USD, SO_LINE_ITEM_TOTAL_COST_CALC]
Index: []


In [130]:
# add a column for warranty source for no charge
df_nocharge['WARRANTY_SOURCE'] = 'NO_CHARGE'

In [131]:
# formatting
df_nocharge['SO_ID'] = df_nocharge['SO_ID'].astype('Int32')
df_nocharge['SO_LINE_TOTAL_INVOICED_QTY'] = pd.to_numeric(df_nocharge['SO_LINE_TOTAL_INVOICED_QTY'], errors='coerce').fillna(0)
df_nocharge['SO_LINE_ITEM_STD_COST_USD'] = pd.to_numeric(df_nocharge['SO_LINE_ITEM_STD_COST_USD'], errors='coerce').fillna(0)

# convert to datetime format
df_nocharge['DATESUBMITTED'] = pd.to_datetime(df_nocharge['DATESUBMITTED'], format='%Y%m%d', errors='coerce')

In [132]:
# add columns to match d2 data
df_nocharge['CLAIMID'] = 'NC_' + df_nocharge['SO_ID'].astype(str)

df_nocharge['CLAIMPARTID'] = 0
df_nocharge['CLAIMPARTID'] = df_nocharge['CLAIMPARTID'].astype('int32')

df_nocharge['PARTQTY'] = df_nocharge['SO_LINE_TOTAL_INVOICED_QTY']

In [133]:
# create product key for future merge with Product Table
df_nocharge['PRODUCT_KEY'] = df_nocharge['COMPANY_ID'].astype(str) + '-' + df_nocharge['PARTNUMBER'].astype(str)

# make sure this is formatted exactly to ensure merging works as intended
df_nocharge['PRODUCT_KEY'] = df_nocharge['PRODUCT_KEY'].str.strip().str.upper().astype(str)

In [134]:
# add column for warranty cost
df_nocharge['NO_CHARGE_COST'] = df_nocharge['SO_LINE_ITEM_TOTAL_COST_CALC']

In [135]:
# merge with company_id table
df_nocharge = pd.merge(df_nocharge,df_company_active, on='COMPANY_ID', how='left')

In [136]:
df_nocharge.head()

,SO_ID,PARTNUMBER,COMPANY_ID,SERVICINGCOMPANY,DATESUBMITTED,ORDERDATE,SHIPMENT_DATE,SO_LINE_TOTAL_INVOICED_QTY,SO_LINE_ITEM_STD_COST_USD,SO_LINE_ITEM_TOTAL_COST_CALC,WARRANTY_SOURCE,CLAIMID,CLAIMPARTID,PARTQTY,PRODUCT_KEY,NO_CHARGE_COST,COMPANY_IS_ACTIVE_FLAG,COMPANY_STATE_CD,COMPANY_REPORTED_NAME,COUNTRY
0,100001,FDXLICB1930,6,Randy's Pool Repair & Service,2020-06-16,2020-06-16,None,1.00,27.21,27.21,NO_CHARGE,NC_100001,0,1.00,6-FDXLICB1930,27.21,YES,CA,"6 - Hayward Industries Inc., Ca., CA",USA
1,100001,HLX-LOC-DSP,6,Randy's Pool Repair & Service,2020-06-16,2020-06-16,None,1.00,93.17,93.17,NO_CHARGE,NC_100001,0,1.00,6-HLX-LOC-DSP,93.17,YES,CA,"6 - Hayward Industries Inc., Ca., CA",USA
2,100003,HWX20000360297,16,SPL,2020-04-23,2020-04-23,None,0.00,3.40,0.00,NO_CHARGE,NC_100003,0,0.00,16-HWX20000360297,0.00,YES,FRA,"16 - Saint-Vulbas, France",EUROPE
3,100006,H400FDN,6,Cesar's Pool Repair,2020-06-16,2020-06-16,None,1.00,744.13,744.13,NO_CHARGE,NC_100006,0,1.00,6-H400FDN,744.13,YES,CA,"6 - Hayward Industries Inc., Ca., CA",USA
4,100007,W32025ADC,6,Nikki Burns,2020-06-16,2020-06-16,None,1.00,82.03,82.03,NO_CHARGE,NC_100007,0,1.00,6-W32025ADC,82.03,YES,CA,"6 - Hayward Industries Inc., Ca., CA",USA


# Join D2 Data

In [137]:
# Perform the outer join with indicator between df_warranty_claims_parts_all and df_warranty_claims_approved
df_temp = df_warranty_claims_parts_all.merge(
    df_warranty_claims_approved, on='CLAIMID', how='outer', indicator=True)

# Add a column to specify which rows are only in the df_warranty_claims_approved table
df_temp['CLAIM_WITHOUT_PART'] = df_temp['_merge'] == 'right_only'
df_temp['CLAIM_WITHOUT_PART'] = np.where(df_temp['_merge'] == 'right_only', 'YES', 'NO')

# Drop the indicator column as it is no longer needed
df_temp = df_temp.drop(columns=['_merge'])

# Now proceed with the rest of your joins using df_temp
df_d2_all = df_temp.merge(
                df_company_active, on='COMPANY_ID', how='left').merge(
                df_serial, on='SERIALNO', how='left').merge(
                df_invoice_aggregate, on='INVOICEID_SO_ITEM', how='left').merge(
                df_ap_aggregate, on='WEBCLAIMID', how='left').merge(
                df_sales_warranty_aggregate, on='SO_COMPANY_ITEM', how='left')

del df_temp

In [138]:
# confirm claims without parts are coming across
df_d2_all[df_d2_all['CLAIM_WITHOUT_PART']=='YES'].tail(1)

,CLAIMPARTID,CLAIMID,POSITIONID,PARTNUMBER,PARTDESC,PARTQTY,PARTREPLACE,PARTCOVERED,PARTSHIP,DENIALCODE,PARTCONDITION,PARTCOMMENT,DATESUBMITTED,DATEUPDATED,DELETED,SHIPREQUESTED,SHIPORDERNO,PARTNOSHIPPED,SHIPFINISHED,SPIFF,PAYSPIFF,APEXPORTED,INVOICEID,TRACKING_1,TRACKING_2,SHIPMENT_DATE,SHIPMENT_CODE,ORDERDATE,PRESHIPPEDFLAG,REMAINUNIT,PARTSNUMBER,APPROVEDSPIFF,CREDIT,ISCREDITEXPORTED,LISTPRICE,SO_ID,COMPANY_ID,CLAIMSHIPMENTID,WEBCLAIMID,TRACKING_3,TRACKING_4,TRACKING_5,TRACKING_6,TRACKING_7,D2_VERSION,INVOICEID_SO_ITEM,SO_COMPANY_ITEM,SERVICINGID,SERVICINGCOMPANY,SERVICINGCITY,SERVICINGSTATE,SERVICINGZIP,REPAIRTYPE,INSTALLERID,INSTALLERNAME,INSTALLERCITY,INSTALLERSTATE,INSTALLERPHONE,INSTALLATIONDATE,FAILUREDATE,REPAIRDATE,POOLTYPE,PRODUCTTYPE,PRODUCTID,MODELNO,SERIALNO,MOTORSERIALNO,FAILURECODE,COMPONENTCODE,REPAIRCODE,PROBLEMREPORTED,PROBLEMFOUND,REPAIRSPERFORMED,LABOR,SALESACCOMMODATION,SALESPERSON,DATESUBMITTED_FOR_SERVICING,STATUS,DENIALCODE_FOR_SERVICING,COMMENTS,ADMINCOMMENTS,ADDITIONALLABOR,DENIALCODE_LABOR,CHANGESTATUSDATE,SERIAL_VALID,LABOR_RATE,SPOC_ADJ,MISCPART_ADJ,TOTAL_LABOR,TOTAL_OTHER,TOTAL_SPIFF,LABOR_APEXPORTED,SPIFF_APEXPORTED,OTHER_APEXPORTED,BETA,AUTHORIZATIONNUM,TOTAL_TAX,TAX_APEXPORTED,SALESPERSONID,AVSHIP,CLAIM_WITHOUT_PART,COMPANY_IS_ACTIVE_FLAG,COMPANY_STATE_CD,COMPANY_REPORTED_NAME,COUNTRY,DATEMFG,MFGID,BUSID,MFG_MONTH,MFG_YEAR,MFG_YEAR_MONTH,INV_NUM,COMPANY_ID_FOR_INVOICE,SO_ID_FOR_INVOICE,INV_LINE_ITEM_ID,INV_SHIP_TO_STATE_CD,INV_COUNTRY_CD,ACTUAL_SHIPMENT_DATE,INV_SHIP_TO_ACCT_NAME,INV_LINE_INVOICED_QTY,INV_LINE_AMT_USD,SPIFF_PAID,LABOR_PAID,MISC_PAID,SO_ID_FOR_SO,COMPANY_ID_FOR_SO,SO_LINE_ITEM_ID,SO_DATE,SO_TYPE_CD,SO_TYPE_DESC,SO_SHIP_TO_STATE_CD,SO_SHIP_TO_COUNTRY_CD,SO_SHIP_TO_ACCT_NAME,SO_LINE_TOTAL_INVOICED_QTY,SO_LINE_ITEM_STD_COST_USD,SO_LINE_ITEM_TOTAL_COST_CALC
1440573,NaN,1225316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135123-99,"ACDC POOL WORKS, INC.",Coconut Creek,FL,33073,1.00,104684,Star-Lite Pools,POMPANO BEACH,FL,954-747-3377,2022-09-01,2024-07-18,2024-07-25,I,11,764.00,HLBASE,3K22033171584,,,,,,,,0.00,False,,2024-07-26 09:53:11.447,1.00,,,,1.00,,2024-07-26 13:27:35,True,95.00,0.00,NaN,95.00,0.00,0.00,95.00,0.00,0.00,False,,NaN,NaN,0.00,False,YES,NaN,NaN,NaN,NaN,2022-02-03,11.00,9.00,2,2022,2022_02,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
# update columns so that claims marked as [CLAIM_WITHOUT_PART = 'yes'] match D2 parts claims
condition = df_d2_all['CLAIM_WITHOUT_PART'] == 'YES'
df_d2_all.loc[condition, 'CLAIMPARTID'] = 0
df_d2_all.loc[condition, 'DATESUBMITTED'] = df_d2_all.loc[condition, 'DATESUBMITTED_FOR_SERVICING']
df_d2_all.loc[condition, 'PARTQTY'] = 0

In [140]:
# Create PRODUCT_KEY column with conditional logic 
df_d2_all['PRODUCT_KEY'] = (
    np.where(
        df_d2_all['COMPANY_ID'].notna(), 
        df_d2_all['COMPANY_ID'].astype(str), 
        np.where(
            df_d2_all['COMPANY_ID_FOR_SO'].notna(),
            df_d2_all['COMPANY_ID_FOR_SO'].astype(str),
            df_d2_all['COMPANY_ID_FOR_INVOICE'].astype(str)
        )
    ) + '-' + df_d2_all['PARTNUMBER'].str.strip().str.upper()
).astype(str)

In [141]:
# free up memory

# create a new, contiguous block of memory, defragmenting the DataFrame
df_d2_approved = df_d2_all[df_d2_all['STATUS'].isin([1,5]) ].copy()

# free memory
del df_d2_all

In [142]:
# add a column for warranty source
df_d2_approved['WARRANTY_SOURCE'] = 'D2'

In [143]:
# create calc to determine how many line items are in each claim
df_d2_approved['CLAIMID_LINE_ITEMS'] = df_d2_approved.groupby('CLAIMID')['CLAIMID'].transform('count')
df_d2_approved.sort_values('TOTAL_LABOR', ascending=False).head()

,CLAIMPARTID,CLAIMID,POSITIONID,PARTNUMBER,PARTDESC,PARTQTY,PARTREPLACE,PARTCOVERED,PARTSHIP,DENIALCODE,PARTCONDITION,PARTCOMMENT,DATESUBMITTED,DATEUPDATED,DELETED,SHIPREQUESTED,SHIPORDERNO,PARTNOSHIPPED,SHIPFINISHED,SPIFF,PAYSPIFF,APEXPORTED,INVOICEID,TRACKING_1,TRACKING_2,SHIPMENT_DATE,SHIPMENT_CODE,ORDERDATE,PRESHIPPEDFLAG,REMAINUNIT,PARTSNUMBER,APPROVEDSPIFF,CREDIT,ISCREDITEXPORTED,LISTPRICE,SO_ID,COMPANY_ID,CLAIMSHIPMENTID,WEBCLAIMID,TRACKING_3,TRACKING_4,TRACKING_5,TRACKING_6,TRACKING_7,D2_VERSION,INVOICEID_SO_ITEM,SO_COMPANY_ITEM,SERVICINGID,SERVICINGCOMPANY,SERVICINGCITY,SERVICINGSTATE,SERVICINGZIP,REPAIRTYPE,INSTALLERID,INSTALLERNAME,INSTALLERCITY,INSTALLERSTATE,INSTALLERPHONE,INSTALLATIONDATE,FAILUREDATE,REPAIRDATE,POOLTYPE,PRODUCTTYPE,PRODUCTID,MODELNO,SERIALNO,MOTORSERIALNO,FAILURECODE,COMPONENTCODE,REPAIRCODE,PROBLEMREPORTED,PROBLEMFOUND,REPAIRSPERFORMED,LABOR,SALESACCOMMODATION,SALESPERSON,DATESUBMITTED_FOR_SERVICING,STATUS,DENIALCODE_FOR_SERVICING,COMMENTS,ADMINCOMMENTS,ADDITIONALLABOR,DENIALCODE_LABOR,CHANGESTATUSDATE,SERIAL_VALID,LABOR_RATE,SPOC_ADJ,MISCPART_ADJ,TOTAL_LABOR,TOTAL_OTHER,TOTAL_SPIFF,LABOR_APEXPORTED,SPIFF_APEXPORTED,OTHER_APEXPORTED,BETA,AUTHORIZATIONNUM,TOTAL_TAX,TAX_APEXPORTED,SALESPERSONID,AVSHIP,CLAIM_WITHOUT_PART,COMPANY_IS_ACTIVE_FLAG,COMPANY_STATE_CD,COMPANY_REPORTED_NAME,COUNTRY,DATEMFG,MFGID,BUSID,MFG_MONTH,MFG_YEAR,MFG_YEAR_MONTH,INV_NUM,COMPANY_ID_FOR_INVOICE,SO_ID_FOR_INVOICE,INV_LINE_ITEM_ID,INV_SHIP_TO_STATE_CD,INV_COUNTRY_CD,ACTUAL_SHIPMENT_DATE,INV_SHIP_TO_ACCT_NAME,INV_LINE_INVOICED_QTY,INV_LINE_AMT_USD,SPIFF_PAID,LABOR_PAID,MISC_PAID,SO_ID_FOR_SO,COMPANY_ID_FOR_SO,SO_LINE_ITEM_ID,SO_DATE,SO_TYPE_CD,SO_TYPE_DESC,SO_SHIP_TO_STATE_CD,SO_SHIP_TO_COUNTRY_CD,SO_SHIP_TO_ACCT_NAME,SO_LINE_TOTAL_INVOICED_QTY,SO_LINE_ITEM_STD_COST_USD,SO_LINE_ITEM_TOTAL_COST_CALC,PRODUCT_KEY,WARRANTY_SOURCE,CLAIMID_LINE_ITEMS
1313570,0.00,1110596,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-26 21:29:34.200,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,249246,"Ideal Contracting, LLC",Monroe,CT,06468,1.00,249246,"Ideal Contracting, LLC",MONROE,CT,203-994-3714,2018-05-15,2023-05-30,2023-05-30,I,6,939.00,LMCUS11050,xxxxxxxxxxxx,,,,,,,,0.00,False,LESLIE'S INSTALLATION,2023-07-26 21:29:34.200,1.00,,Art Soucy is a Hayward Board member and these ...,Per Mike Huppert,1.00,,2023-07-27 10:03:52,False,"3,352.00",0.00,NaN,"3,352.00",0.00,0.00,"3,352.00",0.00,0.00,False,AUTH-03810,NaN,NaN,0.00,False,YES,NaN,NaN,NaN,NaN,NaT,NaN,NaN,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,D2,1
1102904,0.00,898520,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-20 15:33:42.817,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140989-99,Beebe Mechanical,THUNDER BAY,ON,P7C 3R3,1.00,113666-99,Craig Plumbing Centre,Thunder Bay,ON,8076212027,2019-09-01,2020-09-04,2021-05-20,A,4,414.00,H135ID1,21132008103426001,,,,,,,,0.00,False,,2021-05-20 15:33:42.817,1.00,D22,As quoted & approved by Hayward Service Manage...,Approved by Guy to pay for installation of new...,1.00,,2021-05-27 13:10:32,True,"2,777.00",0.00,NaN,"2,777.00",0.00,0.00,"2,777.00",0.00,0.00,False,,361.01,361.01,0.00,False,YES,NaN,NaN,NaN,NaN,2020-08-12,6.00,1.00,8,2020,2020_08,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,D2,1
547391,"531,546.00",399094,2.00,FDXLIGN1930,IGNITOR,1.00,True,True,True,,None,None,2013-04-20 11:32:52.137,NaT,False,True,7-247577,FDXLIGN1930,False,5.00,True,True,70143981,009689440090362,,2013-04-24,579.00,2013-04-23,False,0,None,5.00,0.00,False,NaN,247577,7,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Version_0,70143981_247577_FDXLIGN1930,247577_7.0_FDXLIGN1930,115376-99,Entretien Universel,Montreal,QC,H1B 3X9,1.00,,,,,,2012-09-14,2013-03-13,2013-03-29

In [144]:
# confirm CLAIMID_LINE_ITEMS column is working correctly
df_d2_approved[df_d2_approved['CLAIMID'] == 180876]

,CLAIMPARTID,CLAIMID,POSITIONID,PARTNUMBER,PARTDESC,PARTQTY,PARTREPLACE,PARTCOVERED,PARTSHIP,DENIALCODE,PARTCONDITION,PARTCOMMENT,DATESUBMITTED,DATEUPDATED,DELETED,SHIPREQUESTED,SHIPORDERNO,PARTNOSHIPPED,SHIPFINISHED,SPIFF,PAYSPIFF,APEXPORTED,INVOICEID,TRACKING_1,TRACKING_2,SHIPMENT_DATE,SHIPMENT_CODE,ORDERDATE,PRESHIPPEDFLAG,REMAINUNIT,PARTSNUMBER,APPROVEDSPIFF,CREDIT,ISCREDITEXPORTED,LISTPRICE,SO_ID,COMPANY_ID,CLAIMSHIPMENTID,WEBCLAIMID,TRACKING_3,TRACKING_4,TRACKING_5,TRACKING_6,TRACKING_7,D2_VERSION,INVOICEID_SO_ITEM,SO_COMPANY_ITEM,SERVICINGID,SERVICINGCOMPANY,SERVICINGCITY,SERVICINGSTATE,SERVICINGZIP,REPAIRTYPE,INSTALLERID,INSTALLERNAME,INSTALLERCITY,INSTALLERSTATE,INSTALLERPHONE,INSTALLATIONDATE,FAILUREDATE,REPAIRDATE,POOLTYPE,PRODUCTTYPE,PRODUCTID,MODELNO,SERIALNO,MOTORSERIALNO,FAILURECODE,COMPONENTCODE,REPAIRCODE,PROBLEMREPORTED,PROBLEMFOUND,REPAIRSPERFORMED,LABOR,SALESACCOMMODATION,SALESPERSON,DATESUBMITTED_FOR_SERVICING,STATUS,DENIALCODE_FOR_SERVICING,COMMENTS,ADMINCOMMENTS,ADDITIONALLABOR,DENIALCODE_LABOR,CHANGESTATUSDATE,SERIAL_VALID,LABOR_RATE,SPOC_ADJ,MISCPART_ADJ,TOTAL_LABOR,TOTAL_OTHER,TOTAL_SPIFF,LABOR_APEXPORTED,SPIFF_APEXPORTED,OTHER_APEXPORTED,BETA,AUTHORIZATIONNUM,TOTAL_TAX,TAX_APEXPORTED,SALESPERSONID,AVSHIP,CLAIM_WITHOUT_PART,COMPANY_IS_ACTIVE_FLAG,COMPANY_STATE_CD,COMPANY_REPORTED_NAME,COUNTRY,DATEMFG,MFGID,BUSID,MFG_MONTH,MFG_YEAR,MFG_YEAR_MONTH,INV_NUM,COMPANY_ID_FOR_INVOICE,SO_ID_FOR_INVOICE,INV_LINE_ITEM_ID,INV_SHIP_TO_STATE_CD,INV_COUNTRY_CD,ACTUAL_SHIPMENT_DATE,INV_SHIP_TO_ACCT_NAME,INV_LINE_INVOICED_QTY,INV_LINE_AMT_USD,SPIFF_PAID,LABOR_PAID,MISC_PAID,SO_ID_FOR_SO,COMPANY_ID_FOR_SO,SO_LINE_ITEM_ID,SO_DATE,SO_TYPE_CD,SO_TYPE_DESC,SO_SHIP_TO_STATE_CD,SO_SHIP_TO_COUNTRY_CD,SO_SHIP_TO_ACCT_NAME,SO_LINE_TOTAL_INVOICED_QTY,SO_LINE_ITEM_STD_COST_USD,SO_LINE_ITEM_TOTAL_COST_CALC,PRODUCT_KEY,WARRANTY_SOURCE,CLAIMID_LINE_ITEMS
252238,"245,551.00",180876,1.00,IDXL2BWR1932,BLOWER,1.00,True,True,True,,None,None,2009-05-31 14:55:34.393,2009-06-14 14:00:54.720,False,True,1-369747,IDXL2BWR1932,False,35.96,True,True,11541637,446269074682750,446269074682804,2009-06-26,579.00,2009-06-25,False,0,None,35.96,NaN,False,NaN,369747,1,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Version_0,11541637_369747_IDXL2BWR1932,369747_1.0_IDXL2BWR1932,106257,Buck's Pools LLC,New Palestine,IN,46163,1.00,,Blue Haven Pools & Spas,Indianapolis,IN,317-209-1400,2007-06-01,2009-05-17,2009-05-19,I,4,22.00,H250IDL2,21130602106674001,,SH,BN,,heater not staying on. It starts runs for a fe...,Heater had several items that were being replc...,Burners were warped and the exchanger had to b...,0.00,False,,2009-05-31 14:55:34.300,5.00,,The heat exchanger and pressure switch were pr...,Ok to pay labor per Tom Wiley,1.00,,2009-06-24 17:03:33,True,75.00,0.00,NaN,75.00,0.00,353.94,75.00,353.94,0.00,False,None,NaN,NaN,NaN,None,NO,YES,NC,"1 - Clemmons, NC Pool",USA,2006-02-23,6.00,4.00,2,2006,2006_02,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-IDXL2BWR1932,D2,9
252239,"245,552.00",180876,2.00,IDXL2ICB1931,INTIGRATED CONTROLL BOARD,1.00,True,True,True,,None,None,2009-05-31 14:55:34.487,2009-06-14 14:00:54.817,False,True,1-369747,IDXL2ICB1931,False,19.35,True,True,11541637,446269074682750,446269074682804,2009-06-26,579.00,2009-06-25,False,0,None,19.35,NaN,False,NaN,369747,1,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Version_0,11541637_369747_IDXL2ICB1931,369747_1.0_IDXL2ICB1931,106257,Buck's Pools LLC,New Palestine,IN,46163,1.00,,Blue Haven Pools & Spas,Indianapolis,IN,317-209-1400,2007-06-01,2009-05-17,2009-05-19,I,4,22.00,H250IDL2,21130602106674001,,SH,BN,,heater not staying on. It starts runs for a fe...,Heater had several items that were being replc...,Burners were warped and the exchanger had to b...,0.00,False,,2009-05-31 14:55:34.300,5.00,,The heat exchanger and pressure switch were pr...,Ok to pay labor per Tom Wiley,1.00,,2009-06-24 17:03:33,True,75.00,0.00,NaN,75.00,0.00,353.94,75.00,353.94,0.00,False,None,NaN,NaN,Na

In [145]:
# add adjustment columns to allocate total costs line by line
# for example, TOTAL_LABOR is a cost for the entire claimid while LABOR_ADJ allocates labor for each line
df_d2_approved['LABOR_ADJ'] = df_d2_approved['TOTAL_LABOR'] / df_d2_approved['CLAIMID_LINE_ITEMS']
df_d2_approved['MISC_ADJ'] = df_d2_approved['MISC_PAID'] / df_d2_approved['CLAIMID_LINE_ITEMS']

# if 'PAYSPIFF' = True then it runs calc, If False it sets it to 0
df_d2_approved['SPIFF_ADJ'] = np.where(df_d2_approved['PAYSPIFF'], 
                                       df_d2_approved['SPIFF'] * df_d2_approved['PARTQTY'], 
                                       0)

In [146]:
# confirm adjustment columns work
df_d2_approved[df_d2_approved['CLAIMID'] == 180876]

,CLAIMPARTID,CLAIMID,POSITIONID,PARTNUMBER,PARTDESC,PARTQTY,PARTREPLACE,PARTCOVERED,PARTSHIP,DENIALCODE,PARTCONDITION,PARTCOMMENT,DATESUBMITTED,DATEUPDATED,DELETED,SHIPREQUESTED,SHIPORDERNO,PARTNOSHIPPED,SHIPFINISHED,SPIFF,PAYSPIFF,APEXPORTED,INVOICEID,TRACKING_1,TRACKING_2,SHIPMENT_DATE,SHIPMENT_CODE,ORDERDATE,PRESHIPPEDFLAG,REMAINUNIT,PARTSNUMBER,APPROVEDSPIFF,CREDIT,ISCREDITEXPORTED,LISTPRICE,SO_ID,COMPANY_ID,CLAIMSHIPMENTID,WEBCLAIMID,TRACKING_3,TRACKING_4,TRACKING_5,TRACKING_6,TRACKING_7,D2_VERSION,INVOICEID_SO_ITEM,SO_COMPANY_ITEM,SERVICINGID,SERVICINGCOMPANY,SERVICINGCITY,SERVICINGSTATE,SERVICINGZIP,REPAIRTYPE,INSTALLERID,INSTALLERNAME,INSTALLERCITY,INSTALLERSTATE,INSTALLERPHONE,INSTALLATIONDATE,FAILUREDATE,REPAIRDATE,POOLTYPE,PRODUCTTYPE,PRODUCTID,MODELNO,SERIALNO,MOTORSERIALNO,FAILURECODE,COMPONENTCODE,REPAIRCODE,PROBLEMREPORTED,PROBLEMFOUND,REPAIRSPERFORMED,LABOR,SALESACCOMMODATION,SALESPERSON,DATESUBMITTED_FOR_SERVICING,STATUS,DENIALCODE_FOR_SERVICING,COMMENTS,ADMINCOMMENTS,ADDITIONALLABOR,DENIALCODE_LABOR,CHANGESTATUSDATE,SERIAL_VALID,LABOR_RATE,SPOC_ADJ,MISCPART_ADJ,TOTAL_LABOR,TOTAL_OTHER,TOTAL_SPIFF,LABOR_APEXPORTED,SPIFF_APEXPORTED,OTHER_APEXPORTED,BETA,AUTHORIZATIONNUM,TOTAL_TAX,TAX_APEXPORTED,SALESPERSONID,AVSHIP,CLAIM_WITHOUT_PART,COMPANY_IS_ACTIVE_FLAG,COMPANY_STATE_CD,COMPANY_REPORTED_NAME,COUNTRY,DATEMFG,MFGID,BUSID,MFG_MONTH,MFG_YEAR,MFG_YEAR_MONTH,INV_NUM,COMPANY_ID_FOR_INVOICE,SO_ID_FOR_INVOICE,INV_LINE_ITEM_ID,INV_SHIP_TO_STATE_CD,INV_COUNTRY_CD,ACTUAL_SHIPMENT_DATE,INV_SHIP_TO_ACCT_NAME,INV_LINE_INVOICED_QTY,INV_LINE_AMT_USD,SPIFF_PAID,LABOR_PAID,MISC_PAID,SO_ID_FOR_SO,COMPANY_ID_FOR_SO,SO_LINE_ITEM_ID,SO_DATE,SO_TYPE_CD,SO_TYPE_DESC,SO_SHIP_TO_STATE_CD,SO_SHIP_TO_COUNTRY_CD,SO_SHIP_TO_ACCT_NAME,SO_LINE_TOTAL_INVOICED_QTY,SO_LINE_ITEM_STD_COST_USD,SO_LINE_ITEM_TOTAL_COST_CALC,PRODUCT_KEY,WARRANTY_SOURCE,CLAIMID_LINE_ITEMS,LABOR_ADJ,MISC_ADJ,SPIFF_ADJ
252238,"245,551.00",180876,1.00,IDXL2BWR1932,BLOWER,1.00,True,True,True,,None,None,2009-05-31 14:55:34.393,2009-06-14 14:00:54.720,False,True,1-369747,IDXL2BWR1932,False,35.96,True,True,11541637,446269074682750,446269074682804,2009-06-26,579.00,2009-06-25,False,0,None,35.96,NaN,False,NaN,369747,1,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Version_0,11541637_369747_IDXL2BWR1932,369747_1.0_IDXL2BWR1932,106257,Buck's Pools LLC,New Palestine,IN,46163,1.00,,Blue Haven Pools & Spas,Indianapolis,IN,317-209-1400,2007-06-01,2009-05-17,2009-05-19,I,4,22.00,H250IDL2,21130602106674001,,SH,BN,,heater not staying on. It starts runs for a fe...,Heater had several items that were being replc...,Burners were warped and the exchanger had to b...,0.00,False,,2009-05-31 14:55:34.300,5.00,,The heat exchanger and pressure switch were pr...,Ok to pay labor per Tom Wiley,1.00,,2009-06-24 17:03:33,True,75.00,0.00,NaN,75.00,0.00,353.94,75.00,353.94,0.00,False,None,NaN,NaN,NaN,None,NO,YES,NC,"1 - Clemmons, NC Pool",USA,2006-02-23,6.00,4.00,2,2006,2006_02,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-IDXL2BWR1932,D2,9,8.33,NaN,35.96
252239,"245,552.00",180876,2.00,IDXL2ICB1931,INTIGRATED CONTROLL BOARD,1.00,True,True,True,,None,None,2009-05-31 14:55:34.487,2009-06-14 14:00:54.817,False,True,1-369747,IDXL2ICB1931,False,19.35,True,True,11541637,446269074682750,446269074682804,2009-06-26,579.00,2009-06-25,False,0,None,19.35,NaN,False,NaN,369747,1,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Version_0,11541637_369747_IDXL2ICB1931,369747_1.0_IDXL2ICB1931,106257,Buck's Pools LLC,New Palestine,IN,46163,1.00,,Blue Haven Pools & Spas,Indianapolis,IN,317-209-1400,2007-06-01,2009-05-17,2009-05-19,I,4,22.00,H250IDL2,21130602106674001,,SH,BN,,heater not staying on. It starts runs for a fe...,Heater had several items that were being replc...,Burners were warped and the exchanger had to b...,0.00,False,,2009-05-31 14:55:34.300,5.00,,The heat exchanger and pressure switch were pr...,Ok to pay labor per Tom Wiley,1.00,,2009-06-24 17:03:33,True,75.00,0.00,NaN,75.00,0.00,35

In [147]:
# add column for product cost

# first convert data type
df_d2_approved['SO_LINE_ITEM_STD_COST_USD'] = df_d2_approved['SO_LINE_ITEM_STD_COST_USD'].astype(float)

# calc
# setting this equal to zero for now to test overlap between these charges and no charge orders
df_d2_approved['PRODUCT_COST'] = df_d2_approved['SO_LINE_ITEM_STD_COST_USD'] * df_d2_approved['PARTQTY']

# confirm calc worked
df_d2_approved['PRODUCT_COST'].value_counts(dropna=False).sort_values(ascending=False)

PRODUCT_COST
NaN       1008463
118.14      12592
119.22       8656
106.66       7764
112.73       6805
           ...   
16.65           1
3.56            1
6.20            1
5.42            1
14.61           1
Name: count, Length: 10521, dtype: int64

# Append Distributor Credits and No Charge orders

In [148]:
# it is necessary to merge df_distributor_credits with df_invoice in order to get INV_SHIP_TO_ACCT_NAME
# in order to get a unique row with these 2 columns, it is necessary to do a groupby
df_invoice_grouped_by_acct = df_invoice.groupby('INV_NUM')['INV_SHIP_TO_ACCT_NAME'].agg(list).reset_index()

# Drop duplicates, keeping the first/last occurrence
df_invoice_grouped_by_acct_unique = df_invoice.drop_duplicates(subset='INV_NUM', keep='first')  # or keep='last'

In [149]:
# merge distributor credits with invoice to get the INV_SHIP_TO_ACCT_NAME
df_distributor_credits_merged = pd.merge(df_distributor_credits, 
                                         df_invoice_grouped_by_acct_unique[['INV_NUM', 'INV_SHIP_TO_ACCT_NAME']], 
                                         left_on='INVOICEID', right_on='INV_NUM', how='left', suffixes=('','_right'))

In [150]:
df_distributor_credits_merged.drop(columns=['INV_NUM'], inplace=True)

In [151]:
# confirm the merge did not create additional rows and the groupby worked
print(df_distributor_credits_merged.shape)
print(df_distributor_credits.shape)

(100799, 21)
(100799, 20)


In [152]:
df_distributor_credits_merged.head()

,COMPANY_ID,INVOICEID,SO_ID,PRODUCT_KEY,CUSTOMER_SHIP_TO_KEY,INV_DATE,SO_LINE_ITEM_STD_COST_USD,SHIP_TO_CITY,SHIP_TO_STATE,SHIP_TO_COUNTRY,INV_LINE_INVOICED_QTY,INV_LINE_AMT_USD,WARRANTY_SOURCE,CLAIMID,CLAIMPARTID,DATESUBMITTED,PARTQTY,PARTNUMBER,DISTRIBUTOR_CREDIT_COST,COUNTRY,INV_SHIP_TO_ACCT_NAME
0,1,19202309,505528,1-S180T,1-175067-000105,2019-01-10,68.78,PLAINS,PENNSYLVANIA,United States of America,1,211.75,DISTRIBUTOR_CREDIT,DC_505528,0,2019-01-10,1,S180T,211.75,USA,"SCP DISTRIBUTORS, LLC"
1,1,19240427,760766,1-SP1580X15,1-175067-000007,2021-06-18,62.29,NASHVILLE,TENNESSEE,United States of America,1,203.06,DISTRIBUTOR_CREDIT,DC_760766,0,2021-06-18,1,SP1580X15,203.06,USA,"SCP DISTRIBUTORS, LLC"
2,1,19259448,868608,1-SP2670007X10,1-115600-000000,2022-09-15,203.19,SAGINAW,MICHIGAN,United States of America,1,478.33,DISTRIBUTOR_CREDIT,DC_868608,0,2022-09-15,1,SP2670007X10,478.33,USA,"CINDERELLA, INC."
3,1,19240412,760785,1-SP0607U,1-179320-000000,2021-06-18,23.31,LOUISVILLE,KENTUCKY,United States of America,1,79.46,DISTRIBUTOR_CREDIT,DC_760785,0,2021-06-18,1,SP0607U,79.46,USA,RECREONICS/SWIMMING POOL
4,1,19240411,760316,1-SP2670010X15,1-100009-000002,2021-06-18,125.22,PERRYSBURG,OHIO,United States of America,1,429.18,DISTRIBUTOR_CREDIT,DC_760316,0,2021-06-18,1,SP2670010X15,429.18,USA,QUALITY POOL SUPPLY


In [153]:
# join d2 data, distributor credit, and no charge orders together
df_d2_dc_join = pd.concat([
                           df_d2_approved, 
                           df_distributor_credits_merged, 
                           df_nocharge
                          ], ignore_index=True) # ignore index resets index

C:\Users\mherndon\AppData\Local\Temp\ipykernel_13300\4182990212.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_d2_dc_join = pd.concat([


In [154]:
# merge product table
df_warranty_all = df_d2_dc_join.merge(df_product, on='PRODUCT_KEY', how='left')
df_warranty_all.tail()

,CLAIMPARTID,CLAIMID,POSITIONID,PARTNUMBER,PARTDESC,PARTQTY,PARTREPLACE,PARTCOVERED,PARTSHIP,DENIALCODE,PARTCONDITION,PARTCOMMENT,DATESUBMITTED,DATEUPDATED,DELETED,SHIPREQUESTED,SHIPORDERNO,PARTNOSHIPPED,SHIPFINISHED,SPIFF,PAYSPIFF,APEXPORTED,INVOICEID,TRACKING_1,TRACKING_2,SHIPMENT_DATE,SHIPMENT_CODE,ORDERDATE,PRESHIPPEDFLAG,REMAINUNIT,PARTSNUMBER,APPROVEDSPIFF,CREDIT,ISCREDITEXPORTED,LISTPRICE,SO_ID,COMPANY_ID,CLAIMSHIPMENTID,WEBCLAIMID,TRACKING_3,TRACKING_4,TRACKING_5,TRACKING_6,TRACKING_7,D2_VERSION,INVOICEID_SO_ITEM,SO_COMPANY_ITEM,SERVICINGID,SERVICINGCOMPANY,SERVICINGCITY,SERVICINGSTATE,SERVICINGZIP,REPAIRTYPE,INSTALLERID,INSTALLERNAME,INSTALLERCITY,INSTALLERSTATE,INSTALLERPHONE,INSTALLATIONDATE,FAILUREDATE,REPAIRDATE,POOLTYPE,PRODUCTTYPE,PRODUCTID,MODELNO,SERIALNO,MOTORSERIALNO,FAILURECODE,COMPONENTCODE,REPAIRCODE,PROBLEMREPORTED,PROBLEMFOUND,REPAIRSPERFORMED,LABOR,SALESACCOMMODATION,SALESPERSON,DATESUBMITTED_FOR_SERVICING,STATUS,DENIALCODE_FOR_SERVICING,COMMENTS,ADMINCOMMENTS,ADDITIONALLABOR,DENIALCODE_LABOR,CHANGESTATUSDATE,SERIAL_VALID,LABOR_RATE,SPOC_ADJ,MISCPART_ADJ,TOTAL_LABOR,TOTAL_OTHER,TOTAL_SPIFF,LABOR_APEXPORTED,SPIFF_APEXPORTED,OTHER_APEXPORTED,BETA,AUTHORIZATIONNUM,TOTAL_TAX,TAX_APEXPORTED,SALESPERSONID,AVSHIP,CLAIM_WITHOUT_PART,COMPANY_IS_ACTIVE_FLAG,COMPANY_STATE_CD,COMPANY_REPORTED_NAME,COUNTRY,DATEMFG,MFGID,BUSID,MFG_MONTH,MFG_YEAR,MFG_YEAR_MONTH,INV_NUM,COMPANY_ID_FOR_INVOICE,SO_ID_FOR_INVOICE,INV_LINE_ITEM_ID,INV_SHIP_TO_STATE_CD,INV_COUNTRY_CD,ACTUAL_SHIPMENT_DATE,INV_SHIP_TO_ACCT_NAME,INV_LINE_INVOICED_QTY,INV_LINE_AMT_USD,SPIFF_PAID,LABOR_PAID,MISC_PAID,SO_ID_FOR_SO,COMPANY_ID_FOR_SO,SO_LINE_ITEM_ID,SO_DATE,SO_TYPE_CD,SO_TYPE_DESC,SO_SHIP_TO_STATE_CD,SO_SHIP_TO_COUNTRY_CD,SO_SHIP_TO_ACCT_NAME,SO_LINE_TOTAL_INVOICED_QTY,SO_LINE_ITEM_STD_COST_USD,SO_LINE_ITEM_TOTAL_COST_CALC,PRODUCT_KEY,WARRANTY_SOURCE,CLAIMID_LINE_ITEMS,LABOR_ADJ,MISC_ADJ,SPIFF_ADJ,PRODUCT_COST,CUSTOMER_SHIP_TO_KEY,INV_DATE,SHIP_TO_CITY,SHIP_TO_STATE,SHIP_TO_COUNTRY,DISTRIBUTOR_CREDIT_COST,NO_CHARGE_COST,B_AND_B_NAME,ITEM_DESC,ITEM_MAKE_BUY_DESC,GLOBAL_ITEM_MAKE_BUY_DESC,ITEM_STANDARD_COST_USD,ITEM_MANUFACTURING_COMPANY_ID,ITEM_FAMILY_TYPE_DESC,ITEM_FAMILY_CATEGORY_DESC,ITEM_FORECAST_FAMILY_DESC
2036813,0.00,NC_999855,NaN,ECP28 HW-KIT,NaN,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaT,NaN,2021-08-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999855,3,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAS C/O BKK LANFILL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NC,"3 - Clemmons, NC Flow Control",USA,NaT,NaN,NaN,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,2.35,4.71,3-ECP28 HW-KIT,NO_CHARGE,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,4.71,Flow Control,HANDWHEEL KIT FOR ECP2-8 C & S MODELS,BUY,BUY,3.96,3.00,CONTROLS,ELECTRIC ACTUATORS,ACTUATION
2036814,0.00,NC_999859,NaN,AXV622604WHP,NaN,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-16,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaT,NaN,2020-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999859,6,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JAVI'S #2 POOL EQUIPMENT REPAI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,CA,"6 - Hayward Industries Inc., Ca., CA",USA,NaT,NaN,NaN,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,29.01,58.03,6-AXV622604WHP,NO_CHARGE,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,58.03,US Pools,PROPULSION/WING COMBO REBUILD KIT,MAKE,MAKE,29.01,6.00,CLEANERS,REPLACEMENT PARTS,PARTS - SUCTION CLEANERS
2036815,0.00,NC_999860,NaN,SP0527LED30,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-16,NaT,NaN,NaN,NaN,NaN,NaN,N

In [155]:
# confirm merge and join worked for Distributor Credit
df_warranty_all[df_warranty_all['WARRANTY_SOURCE']=='DISTRIBUTOR_CREDIT']['ITEM_FAMILY_CATEGORY_DESC']

1389440                     SAND FILTER
1389441                    SINGLE SPEED
1389442                    SINGLE SPEED
1389443                          NICHES
1389444                    SINGLE SPEED
                       ...             
1490234                    SINGLE SPEED
1490235    PARAMOUNT HYD VLVS & MODULES
1490236                    SINGLE SPEED
1490237                    SINGLE SPEED
1490238               REPLACEMENT PARTS
Name: ITEM_FAMILY_CATEGORY_DESC, Length: 100799, dtype: object

In [156]:
# confirm merge and join worked for No Charge
df_warranty_all[df_warranty_all['WARRANTY_SOURCE']=='NO_CHARGE']['ITEM_FAMILY_CATEGORY_DESC']

1490239           REPLACEMENT PARTS
1490240           REPLACEMENT PARTS
1490241           REPLACEMENT PARTS
1490242                  GAS HEATER
1490243    SUCTION TURBINE CLEANERS
                     ...           
2036813          ELECTRIC ACTUATORS
2036814           REPLACEMENT PARTS
2036815                         LED
2036816                        SALT
2036817           REPLACEMENT PARTS
Name: ITEM_FAMILY_CATEGORY_DESC, Length: 546579, dtype: object

In [157]:
# ensure that labor only claims have a value for ITEM_FAMILY_TYPE_DESC
# requested by Julie 6/3/24
condition = df_warranty_all['CLAIM_WITHOUT_PART'] == 'YES'
df_warranty_all.loc[condition, 'ITEM_FAMILY_TYPE_DESC'] = 'LABOR_ONLY_CLAIM'

In [158]:
# add PRODUCT_TYPE field
# Create PRODUCT_TYPE column with conditional logic
df_warranty_all['PRODUCT_TYPE'] = (
    np.where(
        df_warranty_all['ITEM_FORECAST_FAMILY_DESC'].notna(), 
        df_warranty_all['ITEM_FORECAST_FAMILY_DESC'].astype(str), 
        np.where(
            df_warranty_all['ITEM_FAMILY_TYPE_DESC'].notna(),
            df_warranty_all['ITEM_FAMILY_TYPE_DESC'].astype(str),
            df_warranty_all['ITEM_FAMILY_CATEGORY_DESC'].fillna('NOT_SPECIFIED').astype(str)  # Fill nulls with empty string
        )
    )
)

In [159]:
# format
df_warranty_all['INV_DATE'] = pd.to_datetime(df_warranty_all['INV_DATE'], errors='coerce')

# does every row have a date submitted?
df_warranty_all['DATESUBMITTED'].isnull().value_counts()

# snowflake will error out if column 'CLAIMID' is not formatted all the same
# CLAIMID format has changed from int32 to str due to the disributor credit and no charge prefixes
df_warranty_all['CLAIMID'] = df_warranty_all['CLAIMID'].astype(str)

In [160]:
# add column for warranty cost

# No charge was throwing an error in calc, so this code can force errors to 0
# df_warranty_all['NO_CHARGE_COST'] = pd.to_numeric(df_warranty_all['NO_CHARGE_COST'], errors='coerce').fillna(0)

df_warranty_all['WARRANTY_COST'] = (
                                    #df_warranty_all['PRODUCT_COST'].fillna(0) + 
                                    df_warranty_all['SPIFF_ADJ'].fillna(0) + 
                                    df_warranty_all['LABOR_ADJ'].fillna(0) + 
                                    df_warranty_all['MISC_ADJ'].fillna(0) +
                                    df_warranty_all['DISTRIBUTOR_CREDIT_COST'].fillna(0) +
                                    df_warranty_all['NO_CHARGE_COST'].fillna(0)
)

# confirm calc worked
df_warranty_all['WARRANTY_COST'].value_counts(dropna=False).sort_values(ascending=False)

WARRANTY_COST
0.00        154682
40.00        77662
85.00        51629
75.00        51148
90.00        32616
             ...  
4,060.00         1
194.44           1
574.80           1
537.89           1
4.88             1
Name: count, Length: 117125, dtype: int64

In [161]:
# make sure all part numbers are upper case
df_warranty_all['PARTNUMBER'] = df_warranty_all['PARTNUMBER'].str.upper().astype(str)

In [162]:
# create a new, contiguous block of memory, defragmenting the dataframe
df_warranty_all = df_warranty_all.copy()

In [163]:
# create month and year columns for grouping
# note than distributor credits do not have the 'DATESUBMITTED' field
df_warranty_all['CLAIM_MONTH'] = df_warranty_all['DATESUBMITTED'].dt.month.astype('Int16')
df_warranty_all['CLAIM_YEAR'] = df_warranty_all['DATESUBMITTED'].dt.year.astype('Int16')
df_warranty_all['CLAIM_YEAR_MONTH'] = df_warranty_all['CLAIM_YEAR'].astype(str) + '_' + df_warranty_all['CLAIM_MONTH'].astype(str).str.zfill(2)
df_warranty_all['INV_MONTH'] = df_warranty_all['INV_DATE'].dt.month.astype('Int16')
df_warranty_all['INV_YEAR'] = df_warranty_all['INV_DATE'].dt.year.astype('Int16')

In [164]:
# format the dates so snowflake will recognize the format when uploading
df_warranty_all['DATESUBMITTED'] = pd.to_datetime(df_warranty_all['DATESUBMITTED'], errors='coerce')
df_warranty_all['SHIPMENT_DATE'] = pd.to_datetime(df_warranty_all['SHIPMENT_DATE'], errors='coerce')
df_warranty_all['ORDERDATE'] = pd.to_datetime(df_warranty_all['ORDERDATE'], errors='coerce')
df_warranty_all['INSTALLATIONDATE'] = pd.to_datetime(df_warranty_all['INSTALLATIONDATE'], errors='coerce')
df_warranty_all['FAILUREDATE'] = pd.to_datetime(df_warranty_all['FAILUREDATE'], errors='coerce')
df_warranty_all['REPAIRDATE'] = pd.to_datetime(df_warranty_all['REPAIRDATE'], errors='coerce')
df_warranty_all['DATESUBMITTED_FOR_SERVICING'] = pd.to_datetime(df_warranty_all['DATESUBMITTED_FOR_SERVICING'], errors='coerce')
df_warranty_all['DATEUPDATED'] = pd.to_datetime(df_warranty_all['DATEUPDATED'], errors='coerce')
df_warranty_all['CHANGESTATUSDATE'] = pd.to_datetime(df_warranty_all['CHANGESTATUSDATE'], errors='coerce')

df_warranty_all['DATESUBMITTED'] = df_warranty_all['DATESUBMITTED'].dt.date
df_warranty_all['SHIPMENT_DATE'] = df_warranty_all['SHIPMENT_DATE'].dt.date
df_warranty_all['ORDERDATE'] = df_warranty_all['ORDERDATE'].dt.date
df_warranty_all['INSTALLATIONDATE'] = df_warranty_all['INSTALLATIONDATE'].dt.date
df_warranty_all['FAILUREDATE'] = df_warranty_all['FAILUREDATE'].dt.date
df_warranty_all['REPAIRDATE'] = df_warranty_all['REPAIRDATE'].dt.date
df_warranty_all['DATESUBMITTED_FOR_SERVICING'] = df_warranty_all['DATESUBMITTED_FOR_SERVICING'].dt.date
df_warranty_all['DATEUPDATED'] = df_warranty_all['DATEUPDATED'].dt.date
df_warranty_all['CHANGESTATUSDATE'] = df_warranty_all['CHANGESTATUSDATE'].dt.date

In [165]:
# Assign country when it is blank or unknown

# Condition to check if 'COUNTRY' is blank
country_not_specified = (df_warranty_all['COUNTRY'].isnull()) | (df_warranty_all['COUNTRY'] == 'UNKNOWN')

# Apply 'determine_country' only to rows where 'COUNTRY' is blank
df_warranty_all.loc[country_not_specified, 'COUNTRY'] = df_warranty_all.loc[country_not_specified, 'SERVICINGSTATE'].apply(
                                                    determine_country)

In [166]:
# check which countries are present
df_warranty_all['COUNTRY'].value_counts(dropna=False)

COUNTRY
USA              1898324
CANADA            105144
EUROPE             30615
LATIN AMERICA       1382
UNKNOWN              675
AUSTRALIA            490
ASIA                  85
MIDDLE EAST           73
AFRICA                29
Other                  1
Name: count, dtype: int64

In [167]:
# check which countries are present (percentages)
country_percentages = df_warranty_all['COUNTRY'].value_counts(normalize=True, dropna=False) * 100
country_percentages

COUNTRY
USA             93.20
CANADA           5.16
EUROPE           1.50
LATIN AMERICA    0.07
UNKNOWN          0.03
AUSTRALIA        0.02
ASIA             0.00
MIDDLE EAST      0.00
AFRICA           0.00
Other            0.00
Name: proportion, dtype: float64

In [168]:
# sales accomodations are not included as part of warranty
df_warranty_all = df_warranty_all[df_warranty_all['SALESACCOMMODATION'] != True]

In [169]:
df_warranty_all.shape

(2002732, 166)

In [170]:
df_warranty_all['SALESACCOMMODATION'].value_counts()

SALESACCOMMODATION
False    1355354
Name: count, dtype: int64

# Assign Manufacturing Plant to Warranty Claim

In [171]:
# set MANUF_PLANT
df_warranty_all['MANUF_PLANT_ID'] = np.where(
    df_warranty_all['GLOBAL_ITEM_MAKE_BUY_DESC'] == 'BUY', 
    'PROCURED',
    np.where(
        df_warranty_all['ITEM_MANUFACTURING_COMPANY_ID'].notnull(), 
        df_warranty_all['ITEM_MANUFACTURING_COMPANY_ID'], 
        np.where(
            df_warranty_all['MFGID'].notnull(), 
            df_warranty_all['MFGID'], 
            df_warranty_all['BUSID']
        )
    )
)

In [172]:
# format data type
df_warranty_all['MANUF_PLANT_ID'] = pd.to_numeric(df_warranty_all['MANUF_PLANT_ID'], errors='coerce')

df_warranty_all['MANUF_PLANT_ID'] = df_warranty_all['MANUF_PLANT_ID'].fillna(0).astype('int16')

In [173]:
# join tables
df_warranty_all = df_warranty_all.merge(df_company[['COMPANY_ID', 'COMPANY_REPORTED_NAME']], left_on='MANUF_PLANT_ID', right_on='COMPANY_ID', how='left', suffixes=('','_MANUF'))

In [174]:
# set COMPANY_REPORTED_NAME_MaNUF = 'Procured' if GLOBAL_ITEM_MAKE_BUY_DESC = Buy
df_warranty_all['COMPANY_REPORTED_NAME_MANUF'] = np.where(
    df_warranty_all['GLOBAL_ITEM_MAKE_BUY_DESC'] == 'BUY', 
    'PROCURED',
     df_warranty_all['COMPANY_REPORTED_NAME_MANUF']
)

In [175]:
df_warranty_all.tail()

,CLAIMPARTID,CLAIMID,POSITIONID,PARTNUMBER,PARTDESC,PARTQTY,PARTREPLACE,PARTCOVERED,PARTSHIP,DENIALCODE,PARTCONDITION,PARTCOMMENT,DATESUBMITTED,DATEUPDATED,DELETED,SHIPREQUESTED,SHIPORDERNO,PARTNOSHIPPED,SHIPFINISHED,SPIFF,PAYSPIFF,APEXPORTED,INVOICEID,TRACKING_1,TRACKING_2,SHIPMENT_DATE,SHIPMENT_CODE,ORDERDATE,PRESHIPPEDFLAG,REMAINUNIT,PARTSNUMBER,APPROVEDSPIFF,CREDIT,ISCREDITEXPORTED,LISTPRICE,SO_ID,COMPANY_ID,CLAIMSHIPMENTID,WEBCLAIMID,TRACKING_3,TRACKING_4,TRACKING_5,TRACKING_6,TRACKING_7,D2_VERSION,INVOICEID_SO_ITEM,SO_COMPANY_ITEM,SERVICINGID,SERVICINGCOMPANY,SERVICINGCITY,SERVICINGSTATE,SERVICINGZIP,REPAIRTYPE,INSTALLERID,INSTALLERNAME,INSTALLERCITY,INSTALLERSTATE,INSTALLERPHONE,INSTALLATIONDATE,FAILUREDATE,REPAIRDATE,POOLTYPE,PRODUCTTYPE,PRODUCTID,MODELNO,SERIALNO,MOTORSERIALNO,FAILURECODE,COMPONENTCODE,REPAIRCODE,PROBLEMREPORTED,PROBLEMFOUND,REPAIRSPERFORMED,LABOR,SALESACCOMMODATION,SALESPERSON,DATESUBMITTED_FOR_SERVICING,STATUS,DENIALCODE_FOR_SERVICING,COMMENTS,ADMINCOMMENTS,ADDITIONALLABOR,DENIALCODE_LABOR,CHANGESTATUSDATE,SERIAL_VALID,LABOR_RATE,SPOC_ADJ,MISCPART_ADJ,TOTAL_LABOR,TOTAL_OTHER,TOTAL_SPIFF,LABOR_APEXPORTED,SPIFF_APEXPORTED,OTHER_APEXPORTED,BETA,AUTHORIZATIONNUM,TOTAL_TAX,TAX_APEXPORTED,SALESPERSONID,AVSHIP,CLAIM_WITHOUT_PART,COMPANY_IS_ACTIVE_FLAG,COMPANY_STATE_CD,COMPANY_REPORTED_NAME,COUNTRY,DATEMFG,MFGID,BUSID,MFG_MONTH,MFG_YEAR,MFG_YEAR_MONTH,INV_NUM,COMPANY_ID_FOR_INVOICE,SO_ID_FOR_INVOICE,INV_LINE_ITEM_ID,INV_SHIP_TO_STATE_CD,INV_COUNTRY_CD,ACTUAL_SHIPMENT_DATE,INV_SHIP_TO_ACCT_NAME,INV_LINE_INVOICED_QTY,INV_LINE_AMT_USD,SPIFF_PAID,LABOR_PAID,MISC_PAID,SO_ID_FOR_SO,COMPANY_ID_FOR_SO,SO_LINE_ITEM_ID,SO_DATE,SO_TYPE_CD,SO_TYPE_DESC,SO_SHIP_TO_STATE_CD,SO_SHIP_TO_COUNTRY_CD,SO_SHIP_TO_ACCT_NAME,SO_LINE_TOTAL_INVOICED_QTY,SO_LINE_ITEM_STD_COST_USD,SO_LINE_ITEM_TOTAL_COST_CALC,PRODUCT_KEY,WARRANTY_SOURCE,CLAIMID_LINE_ITEMS,LABOR_ADJ,MISC_ADJ,SPIFF_ADJ,PRODUCT_COST,CUSTOMER_SHIP_TO_KEY,INV_DATE,SHIP_TO_CITY,SHIP_TO_STATE,SHIP_TO_COUNTRY,DISTRIBUTOR_CREDIT_COST,NO_CHARGE_COST,B_AND_B_NAME,ITEM_DESC,ITEM_MAKE_BUY_DESC,GLOBAL_ITEM_MAKE_BUY_DESC,ITEM_STANDARD_COST_USD,ITEM_MANUFACTURING_COMPANY_ID,ITEM_FAMILY_TYPE_DESC,ITEM_FAMILY_CATEGORY_DESC,ITEM_FORECAST_FAMILY_DESC,PRODUCT_TYPE,WARRANTY_COST,CLAIM_MONTH,CLAIM_YEAR,CLAIM_YEAR_MONTH,INV_MONTH,INV_YEAR,MANUF_PLANT_ID,COMPANY_ID_MANUF,COMPANY_REPORTED_NAME_MANUF
2002727,0.00,NC_999855,NaN,ECP28 HW-KIT,NaN,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaT,NaN,2021-08-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999855,3,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAS C/O BKK LANFILL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NC,"3 - Clemmons, NC Flow Control",USA,NaT,NaN,NaN,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,2.35,4.71,3-ECP28 HW-KIT,NO_CHARGE,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,4.71,Flow Control,HANDWHEEL KIT FOR ECP2-8 C & S MODELS,BUY,BUY,3.96,3.00,CONTROLS,ELECTRIC ACTUATORS,ACTUATION,ACTUATION,4.71,8,2021,2021_08,<NA>,<NA>,0,NaN,PROCURED
2002728,0.00,NC_999859,NaN,AXV622604WHP,NaN,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-16,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaT,NaN,2020-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999859,6,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JAVI'S #2 POOL EQUIPMENT REPAI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,CA,"6 - Hayward Industries Inc., Ca., CA",USA,NaT,NaN,NaN,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,29.01,58.03,6-AXV622604WHP,NO_CHARGE,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,58.03,US Pools,PROPULSION/WIN

In [176]:
df_warranty_all['COMPANY_REPORTED_NAME_MANUF'].value_counts(dropna=False)

COMPANY_REPORTED_NAME_MANUF
9 - North Kingstown, RI                 619922
1 - Clemmons, NC Pool                   489978
4 - Wuxi, China                         248906
5 - Nashville, TN                       234713
6 - Hayward Industries Inc., Ca., CA    132395
NaN                                      99150
PROCURED                                 93405
11 - Hayward / Aqua Leader, NC           37395
60 - Phoenix, AZ Pool                    20530
7 - Oakville, Canada                     10214
16 - Saint-Vulbas, France                 9337
8 - Phoenix, AZ Flow Control              3843
3 - Clemmons, NC Flow Control             1605
12 - Goldline, NJ                         1282
23 - IMG, NC                                29
15 - Aquavac Pool Cleaners Corp., FL        27
20 - IMG, AZ                                 1
Name: count, dtype: int64

In [177]:
# test NaN values for COMPANY_REPORTED_NAME_MANUF

test123 = df_warranty_all[df_warranty_all['COMPANY_REPORTED_NAME_MANUF'].isnull() ]
#test123[['MANUF_PLANT', 'MFGID', 'BUSID', 'GLOBAL_ITEM_MAKE_BUY_DESC']].head()
test123.loc[test123['CLAIM_YEAR'] == 2024, ['MANUF_PLANT_ID', 'MFGID', 'BUSID', 'GLOBAL_ITEM_MAKE_BUY_DESC', 'ITEM_MANUFACTURING_COMPANY_ID', 'COMPANY_REPORTED_NAME_MANUF']].tail()
#test123.loc[test123['CLAIM_YEAR'] == 2024].head()

,MANUF_PLANT_ID,MFGID,BUSID,GLOBAL_ITEM_MAKE_BUY_DESC,ITEM_MANUFACTURING_COMPANY_ID,COMPANY_REPORTED_NAME_MANUF
1355090,0,NaN,NaN,NaN,NaN,NaN
1355098,0,NaN,NaN,NaN,NaN,NaN
1355099,0,NaN,NaN,NaN,NaN,NaN
1355122,0,NaN,NaN,NaN,NaN,NaN
1355143,0,NaN,NaN,NaN,NaN,NaN


In [178]:
# free up memory
# del df_distributor_credits_merged
# del df_distributor_credits

# del df_invoice_grouped_by_acct
# del df_invoice_grouped_by_acct_unique
# del df_invoice
# del df_invoice_aggregate

# del df_d2_approved

# del df_product

# del df_company

# del df_nocharge
# del df_nocharge_prelim

# del df_warranty_claims_approved
# del df_warranty_claims_parts_all

# del df_serial

# del df_ap_aggregate

# del df_sales_order
# del df_sales_warranty
# del df_sales_warranty_aggregate

# see if any dataframes are still in memory that should be removed
# %whos

# Write dataframe to snowflake

In [180]:
# Configure a connection
ctx = snowflake.connector.connect(
    user='hidden',
    password='hidden',# need to update password here -------------------------------------------
    account='hidden',
    warehouse='BI_MEDIUM_WH',
    database='FIN_DB',
    schema='BUSINESS_INTELLIGENCE'
)

In [181]:
# write Warranty Data Model to Snowflake

# create a cursor object
cs=ctx.cursor()

# force the database and schema
#cs.execute('USE WAREHOUSE BI_MEDIUM_WH')
cs.execute('USE DATABASE FIN_DB')
cs.execute('USE SCHEMA BUSINESS_INTELLIGENCE')

# give table some name
table='WARRANTY_DATA_MODEL'

# simple version for test
df = df_warranty_all

# create table in snowflake
write_pandas(ctx, df, table, auto_create_table=True, overwrite=True)

# check and query created table
query = f"SELECT count(*) FROM {table};"
count = cs.execute(query).fetchone()
print(f"Snowflake table {table} has {count} rows")


Snowflake table WARRANTY_DATA_MODEL has (2002732,) rows


In [182]:
# view data types that should be in snowflake

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_warranty_all.dtypes)

CLAIMPARTID                             float64
CLAIMID                                  object
POSITIONID                              float64
PARTNUMBER                               object
PARTDESC                                 object
PARTQTY                                 float64
PARTREPLACE                              object
PARTCOVERED                              object
PARTSHIP                                 object
DENIALCODE                               object
PARTCONDITION                            object
PARTCOMMENT                              object
DATESUBMITTED                            object
DATEUPDATED                              object
DELETED                                  object
SHIPREQUESTED                            object
SHIPORDERNO                              object
PARTNOSHIPPED                            object
SHIPFINISHED                             object
SPIFF                                   float64
PAYSPIFF                                

In [183]:
# Write Invoice Agg to Snowflake

# create a cursor object
cs=ctx.cursor()

# force the database and schema
cs.execute('USE DATABASE FIN_DB')
cs.execute('USE SCHEMA BUSINESS_INTELLIGENCE')

# give table some name
table='INVOICE_AGGREGATION_FOR_WARRANTY'

# simple version for test
df = df_invoice_aggregate

# create table in snowflake
write_pandas(ctx, df, table, auto_create_table=True, overwrite=True)

# check and query created table
query = f"SELECT count(*) FROM {table};"
count = cs.execute(query).fetchone()
print(f"Snowflake table {table} has {count} rows")

# Close the cursor and connection
cs.close()
ctx.close()

Snowflake table INVOICE_AGGREGATION_FOR_WARRANTY has (4150581,) rows
